CS598 DL4H - Spring 2023
Dhiraj Tiwary
Dtiwary2@illinois.edu
Group ID: 116
Paper ID: 45
Presentation link: https://youtu.be/k6YI1bnn-fY
Code link: https://github.com/dtiwary2/dl4hspring2023


MIMIC - data processing

In [ ]:
#!/usr/bin/env python3
# INSERT FILE DESCRIPTION

"""
Util functions to run Model. Includes Data loading, etc...
"""
import os
from typing import List, Union

import numpy as np
import pandas as pd

from tqdm import tqdm
import datetime as dt

tqdm.pandas()


def _compute_last_target_id(df: pd.DataFrame, time_col: str = "intime", mode: str = "max") -> pd.DataFrame:
    """Identify last ids given df according to time given by time_col column. Mode determines min or max."""
    if mode == "max":
        time = df[time_col].max()
    elif mode == "min":
        time = df[time_col].min()
    else:
        raise ValueError("mode must be one of ['min', 'max']. Got {}".format(mode))

    last_ids = df[df[time_col] == time]

    return last_ids


def _rows_are_in(df1: pd.DataFrame, df2: pd.DataFrame, matching_columns: Union[List[str], str]) -> pd.DataFrame:
    """
    Checks if values present in row of df1 exist for all columns in df2. Note that this does not necessarily mean
    the whole row of df1 is in df2, but is good enough for application.

    Returns: array of indices indicating the relevant rows of df1.
    """
    if isinstance(matching_columns, str):
        matching_columns = [matching_columns]

    # Iterate through each column
    matching_ids = np.ones(df1.shape[0])
    for col in tqdm(matching_columns):
        col_matching = df1[col].isin(df2[col].values).values  # where df1 col is subset of df2 col
        matching_ids = np.logical_and(matching_ids, col_matching)  # match with columns already looked at

    return matching_ids


def _compute_second_transfer_info(df: pd.DataFrame, time_col, target_cols):
    """
    Given transfer data for a unique id, compute the second transfer as given by time_col.

    return: pd.Series with corresponding second transfer info.
    """
    time_info = df[time_col]
    second_transfer_time = time_info[time_info != time_info.min()].min()

    # Identify second transfer info - can be empty, unique, or repeated instances
    second_transfer = df[df[time_col] == second_transfer_time]

    if second_transfer.empty:
        output = [df.name, df["hadm_id"].iloc[0], df["transfer_id"].iloc[0]] + [np.nan] * (len(target_cols) - 3)
        return pd.Series(data=output, index=target_cols)

    elif second_transfer.shape[0] == 1:
        return pd.Series(data=second_transfer.squeeze().values, index=target_cols)

    else:  # There should be NONE
        print(second_transfer)
        raise ValueError("Something's gone wrong! No expected repeated second transfers with the same time.")


def convert_columns_to_dt(df: pd.DataFrame, columns: Union[str, List[str]]):
    """Convert columns of dataframe to datetime format, as per given"""
    if isinstance(columns, str):
        columns = [columns]

    for col in columns:
        df[col] = pd.to_datetime(df.loc[:, col].values)

    return df


def subsetted_by(df1: pd.DataFrame, df2: pd.DataFrame, matching_columns: Union[List[str], str]) -> pd.DataFrame:
    """
    Subset df1 based on matching_columns, according to values existing in df2.

    Returns: pd.DataFrame subset of df1 for which rows are a subset of df2
    """

    return df1.iloc[_rows_are_in(df1, df2, matching_columns), :]


def endpoint_target_ids(df: pd.DataFrame, identifier: str, time_col: str = "intime", mode: str = "max") -> pd.DataFrame:
    """
    Given identifier target ("id"), compute the endpoint associated with time column.

    Returns: pd.DataFrame with ids and associated endpoint information.
    """
    last_ids = df.groupby(identifier, as_index=False).progress_apply(
        lambda x: _compute_last_target_id(x, time_col=time_col, mode=mode))

    return last_ids.reset_index(drop=True)


def compute_second_transfer(df: pd.DataFrame, identifier: str, time_col: str, target_cols: pd.Index) -> pd.DataFrame:
    """
    Given transfer data represented by unique identifier ("id"), compute the second transfer of the admission.
    Second Transfer defined as second present intime in the date (if multiple, this is flagged). If there are
    no transfers after, then return np.nan. target_cols is the target information columns.

    This function checks the second transfer intime is after outtime of first transfer record.

    Returns: pd.DataFrame with id and associated second transfer information (intime/outtime, unit, etc...)
    """
    second_transfer_info = df.groupby(identifier, as_index=False).progress_apply(
        lambda x: _compute_second_transfer_info(x, time_col, target_cols))

    return second_transfer_info.reset_index(drop=True)


def _has_many_nas(df: pd.DataFrame, targets: Union[List[str], str], min_count: int, min_frac: float) -> bool:
    """
    For a given admission/stay with corresponding vital sign information, return boolean indicating whether low
    missingness conditions are satisfied. These are:
    a) At least min_count observations.
    b) Proportion of missing values smaller than min_frac for ALL targets.

    returns: boolean indicating admission should be kept.
    """
    if isinstance(targets, str):
        targets = [targets]

    has_minimum_counts = df.shape[0] > min_count
    has_less_NA_than_frac = df[targets].isna().sum() <= min_frac * df.shape[0]

    return has_minimum_counts and has_less_NA_than_frac.all()


def remove_adms_high_missingness(df: pd.DataFrame, targets: Union[List[str], str],
                                 identifier: str, min_count: int, min_frac: float) -> pd.DataFrame:
    """
    Given vital sign data, remove admissions with too little information. This is defined as either:
    a) Number of observations smaller than allowed min_count.
    b) Proportion of missing values in ANY of the targets is higher than min_frac.

    Returns: pd.DataFrame - Vital sign data of the same type, except only admissions with enough information are kept.
    """
    output = df.groupby(identifier, as_index=False).filter(
        lambda x: _has_many_nas(x, targets, min_count, min_frac))

    return output.reset_index(drop=True)


def _resample_adm(df: pd.DataFrame, rule: str, time_id: str,
                  time_vars: Union[List[str], str], static_vars: Union[List[str], str]) -> pd.DataFrame:
    """
    For a particular stay with vital sign data as per df, resample trajectory data (subsetted to time_vars),
    according to index given by time_to_end and as defined by rule. It is important that time_to_end decreases
    throughout admissions and hits 0 at the end - this is for resampling purposes.

    Params:
    df: pd.Dataframe, containing trajectory and static data for each admission.
    rule: str, indicates the resampling rule (to be fed to pd.DataFrame.resample())

    static_vars is a list of relevant identifier information

    returns: Resampled admission data. Furthermore, two more info columns are indicated (chartmax and chartmin).
    """
    if isinstance(time_vars, str):
        time_vars = [time_vars]

    if isinstance(static_vars, str):
        static_vars = [static_vars]

    # Add fake observation (with missing values) so that resampling starts at end of admission
    df_inter = df[time_vars + ["time_to_end"]]
    df_inter = df_inter.append(pd.Series(data=[np.nan] * len(time_vars) + [dt.timedelta(seconds=0)],
                                         index=df_inter.columns), ignore_index=True)

    # resample on time_to_end axis
    output = df_inter.sort_values(by="time_to_end", ascending=False).resample(
        on="time_to_end",
        rule=rule, closed="left", label="left").mean()

    # Compute static ids manually and add information about max and min time id values
    output[static_vars] = df[static_vars].iloc[0, :].values
    output[time_id + "_min"] = df[time_id].min()
    output[time_id + "_max"] = df[time_id].max()

    # Reset index to obtain resampled values
    output.index.name = f"sampled_time_to_end({rule})"
    output.reset_index(drop=False, inplace=True)

    return output


def compute_time_to_end(df: pd.DataFrame, id_key: str, time_id: str, end_col: str):
    """
    Compute time to end of admission for a given observation associated with a particular admission id.

    df: pd.DataFrame with trajectory information.
    id_key: str - column of df representing the unique id admission identifier.
    time_id: str - column of df indicating time observations was taken.
    end_col: str - column of df indicating, for each observation, the end time of the corresponding admission.

    returns: sorted pd.DataFrame with an extra column indicating time to end of admission. This will be used for
    resampling.
    """
    df_inter = df.copy()
    df_inter["time_to_end"] = df_inter[end_col] - df_inter[time_id]
    df_inter.sort_values(by=[id_key, "time_to_end"], ascending=[True, False], inplace=True)

    return df_inter


def conversion_to_block(df: pd.DataFrame, id_key: str, rule: str,
                        time_vars: Union[List[str], str], static_vars: Union[List[str], str]) -> pd.DataFrame:
    """
    Given trajectory data over multiple admissions (as specified by id), resample each admission according to time
    until the end of the admission. Resampling according to rule and apply to_time_vars.

    df: pd.DataFrame containing trajectory and static data.
    id_key: str, unique identifier per admission
    rule: str, indicates resampling rule (to be fed to pd.DataFrame.resample())
    time_vars: list of str, indicates columns of df to be resampled.
    static_vars: list of str, indicates columns of df which are static, and therefore not resampled.

    return: Dataframe with resampled vital sign data.
    """
    if "time_to_end" not in df.columns:
        raise ValueError("'time_to_end' not found in columns of dataframe. Run 'compute_time_to_end' function first.")
    assert df[id_key].is_monotonic and df.groupby(id_key).apply(
        lambda x: x["time_to_end"].is_monotonic_decreasing).all()

    # Resample admission according to time_to_end
    output = df.groupby(id_key).progress_apply(lambda x: _resample_adm(x, rule, "time_to_end", time_vars, static_vars))

    return output.reset_index(drop=True)


def convert_to_timedelta(df: pd.DataFrame, *args) -> pd.DataFrame:
    """Convert all given cols of dataframe to timedelta."""
    output = df.copy()
    for arg in args:
        output[arg] = pd.to_timedelta(df.loc[:, arg])

    return output


def _check_all_tables_exist(folder_path: str):
    """TO MOVE TO TEST"""
    try:
        assert os.path.exists(folder_path)
    except Exception:
        raise ValueError("Folder path does not exist - Input {}".format(folder_path))


def select_death_icu_acute(df, admissions_df, timedt):
    """
    Identify outcomes based on severity within the consequent 12 hours:
    a) Death
    b) Entry to ICU Careunit
    c) Transfer to hospital ward
    d) Discharge

    Params:
    - df - transfers dataframe corresponding to a particular admission.
    - timedt - datetime timedelta indicating range window of prediction

    Returns categorical encoding of the corresponding admission.
    Else returns 0,0,0,0 if a mistake is found.
    """
    # Check admission contains only one such row
    assert admissions_df.hadm_id.eq(df.name).sum() <= 1

    # Identify Last observed vitals for corresponding admission
    hadm_information = admissions_df.query("hadm_id==@df.name").iloc[0, :]
    window_start_point = hadm_information.loc["outtime"]

    # First check if death exists
    hadm_information = admissions_df.query("hadm_id==@df.name")
    if not hadm_information.empty and not hadm_information.dod.isna().all():
        time_of_death = hadm_information.dod.min()
        time_from_start_point = (time_of_death - window_start_point)

        # try:
        #     assert time_from_vitals >= dt.timedelta(seconds=0)
        #
        # except AssertionError:
        #     return pd.Series(data=[0, 0, 0, 0, time_of_death], index=["De", "I", "W", "Di", "time"])

        # Check death within time window
        if time_from_start_point < timedt:
            return pd.Series(data=[1, 0, 0, 0, time_of_death], index=["De", "I", "W", "Di", "time"])

    # Otherwise, consider other transfers
    transfers_within_window = df[df["intime"].between(window_start_point, window_start_point + timedt)]

    # Consider icu transfers within window
    icu_cond1 = transfers_within_window.careunit.str.contains("(?i)ICU", na=False)  # regex ignore lowercase
    icu_cond2 = transfers_within_window.careunit.str.contains("(?i)Neuro Stepdown", na=False)
    has_icus = (icu_cond1 | icu_cond2)

    if has_icus.sum() > 0:
        icu_transfers = transfers_within_window[has_icus]
        return pd.Series(data=[0, 1, 0, 0, icu_transfers.intime.min()],
                         index=["De", "I", "W", "Di", "time"])

    # Check to see if discharge has taken
    discharges = transfers_within_window.eventtype.str.contains("discharge", na=False)
    if discharges.sum() > 0:
        return pd.Series(data=[0, 0, 0, 1, transfers_within_window[discharges].intime.min()],
                         index=["De", "I", "W", "Di", "time"]
                         )
    else:
        return pd.Series(data=[0, 0, 1, 0, transfers_within_window.intime.min()],
                         index=["De", "I", "W", "Di", "time"]
                         )


In [ ]:
# Processing script for initial ED admission processing.

import os
import pandas as pd

####################################################
"""

Processing Steps:

1. Compute recorded intime and outimes for each ED admission.
2. Select admissions with ED as the first admission.
3. Remove admissions admitted to special wards, including Partum and Psychiatry. Compute next transfer information.
4. Add patient core information.
5. Remove admissions without triage information.


Other notes.
ROW SUBSETTING COULD BE IMPROVED SOMEHOW
"""

# ------------------------------------ // --------------------------------------
"""
List of variables used for processing which should be fixed.

Data_FD: where the data is saved.
SAVE_FD: folder path of interim data saving.
ID_COLUMNS: identifiers for admissions, patients and hospital stays.
TIME_COLUMNS: list of datetime object columns.
WARDS_TO_REMOVE: list of special wards where patients were transferred to and which represent unique populations. This
list includes Partum and Psychiatry wards, as well as further ED observations, which generally take place when 
the hospital is full.
AGE_LOWERBOUND: minimum age of patients.
PATIENT_INFO: characteristic information for each patient.
NEXT_TRANSFER_INFO: list of important info to keep related to the subsequent transfer from ED.
"""
DATA_FD = "data/MIMIC/"
SAVE_FD = DATA_FD + "interim/"
ID_COLUMNS = ["subject_id", "hadm_id", "stay_id"]
TIME_COLUMNS = ["intime", "outtime", "charttime", "deathtime"]
WARDS_TO_REMOVE = ["Unknown", "Emergency Department", "Obstetrics Postpartum",
                   "Obstetrics Antepartum", "Obstetrics (Postpartum & Antepartum)",
                   "Psychiatry", "Labor & Delivery", "Observation", "Emergency Department Observation"]
AGE_LOWERBOUND = 18
PATIENT_INFO = ["gender", "anchor_age", "anchor_year", "dod"]
NEXT_TRANSFER_INFO = ["transfer_id", "eventtype", "careunit", "intime", "outtime"]

if not os.path.exists(SAVE_FD):
    os.makedirs(SAVE_FD)


# ------------------------------------- // -------------------------------------
def admissions_processing():
    """
    First, Tables are Loaded. We load 4 tables:
    
    - patients_core: from core/patients filepath. This is a dataframe of patient centralised admission information. 
    Cohort information for each patient is computed, as well as a unique id which is consistent across all other tables.
    
    - transfer_core: from core/transfers.csv filepath. This is a dataframe with a list of transfers for each patient.
    Includes admissions to ED, but also transfers to wards in the hospital, ICUs, etc...
    
    - admissions_ed: from ed/edstays.csv filepath. This is a dataframe of patient information indicating relevant
    information for any ED admission.
    
    - triage_ed: from ed/triage.csv filepath. This is a dataframe of patient ED admission indicating triage assessments.
    """

    # Hospital Core
    patients_core = pd.read_csv(DATA_FD + "core/patients.csv", index_col=None, header=0, low_memory=False)
    transfers_core = pd.read_csv(DATA_FD + "core/transfers.csv", index_col=None, header=0, low_memory=False,
                                 parse_dates=["intime", "outtime"])

    # ED Admission
    admissions_ed = pd.read_csv(DATA_FD + "ed/edstays.csv", index_col=None, header=0, low_memory=False,
                                parse_dates=["intime", "outtime"])
    triage_ed = pd.read_csv(DATA_FD + "ed/triage.csv", index_col=None, header=0, low_memory=False)

    # ------------------------------------- // -------------------------------------
    """
    Process Admission data according to multiple steps.
    
    Step 1: Remove double admission counts. Select the latest intime. If there are multiple such intimes, 
    select the last outtime.
    Consider only these admissions.
    """

    # Compute recorded admission intimes and outtimes. Respectively, select latest intime and outtime.
    admissions_intime_ed = utils.endpoint_target_ids(admissions_ed, "subject_id", "intime")
    admissions_outtime_ed = utils.endpoint_target_ids(admissions_intime_ed, "subject_id", "outtime")

    admissions_ed_S1 = utils.subsetted_by(admissions_ed, admissions_outtime_ed,
                                          ["stay_id"])  # last admission information
    admissions_ed_S1.to_csv(SAVE_FD + "admissions_S1.csv", index=True, header=True)

    """
    Identify those admissions where patients were directly sent to Emergency Department, i.e., the first intime
    is in the Emergency Department. 
    Subset to these admissions.
    """
    # Identify first wards for all admissions to hospital
    transfers_first_ward = utils.endpoint_target_ids(transfers_core, "subject_id", "intime", mode="min")
    ed_first_transfer = transfers_first_ward[(transfers_first_ward["eventtype"] == "ED") &
                                             (transfers_first_ward["careunit"] == "Emergency Department")]

    # Subset to admissions with ED as first transfer
    admissions_ed_S2 = utils.subsetted_by(admissions_ed_S1, ed_first_transfer,
                                          ["subject_id", "hadm_id", "intime", "outtime"])
    transfers_ed_S2 = utils.subsetted_by(transfers_core, admissions_ed_S2, ["subject_id", "hadm_id"])
    admissions_ed_S2.to_csv(SAVE_FD + "admissions_S2.csv", index=True, header=True)

    """
    Consider only those admissions for which they did not have a subsequent transfer to a Special ward, which includes
    Partum and Psychiatry wards. The full list of wards is identified in WARDS TO REMOVE
    """
    # Remove admissions transferred to irrelevant wards (Partum, Psychiatry). Furthermore, EDObs is also special.
    # Missing check that second intime is after ED outtime
    transfers_second_ward = utils.compute_second_transfer(transfers_ed_S2, "subject_id", "intime",
                                                          transfers_ed_S2.columns)
    transfers_to_relevant_wards = transfers_second_ward[~ transfers_second_ward.careunit.isin(WARDS_TO_REMOVE)]
    admissions_ed_S3 = utils.subsetted_by(admissions_ed_S2, transfers_to_relevant_wards, ["subject_id", "hadm_id"])

    # ADD patient core information and next Transfer Information.
    patients_S3 = admissions_ed_S3.subject_id.values
    admissions_ed_S3.loc[:, PATIENT_INFO] = patients_core.set_index("subject_id").loc[patients_S3, PATIENT_INFO].values

    for col in NEXT_TRANSFER_INFO:
        admissions_ed_S3.loc[:, "next_" + col] = transfers_to_relevant_wards.set_index("subject_id").loc[
            patients_S3, col].values

    # Compute age and save
    admissions_ed_S3["age"] = admissions_ed_S3.intime.dt.year - admissions_ed_S3["anchor_year"] + admissions_ed_S3[
        "anchor_age"]
    admissions_ed_S3.to_csv(SAVE_FD + "admissions_S3.csv", index=True, header=True)

    """
    Step 4: Patients must have an age older than AGE LOWERBOUND
    """
    # Compute age and Remove below AGE LOWERBOUND
    admissions_ed_S4 = admissions_ed_S3[admissions_ed_S3["age"] >= AGE_LOWERBOUND]
    admissions_ed_S4.to_csv(SAVE_FD + "admissions_S4.csv", index=True, header=True)

    """
    Step 5: Add ESI information, and subset to patients with ESI values and between 2, 3, 4.
    ESI values of 1 and 5 are edge cases (nothing wrong with them, or in extremely critical condition).
    """
    # Compute and remove ESI NAN, ESI 1 and ESI 5 and save
    admissions_ed_S4["ESI"] = triage_ed.set_index("stay_id").loc[admissions_ed_S4.stay_id.values, "acuity"].values
    admissions_ed_S5 = admissions_ed_S4[~ admissions_ed_S4["ESI"].isna()]
    # admissions_ed_S5 = admissions_ed_S5[~ admissions_ed_S5["ESI"].isin([1, 5])]

    # Save data
    admissions_ed_S5.to_csv(SAVE_FD + "admissions_intermediate.csv", index=True, header=True)


In [ ]:
# Processing script for vitals ED admissions.


"""

Run -admissions_processing.py first.

Processing Steps:

1. Identify patients computed from admissions_processing.py cohort.
2. Consider vitals only between intime and outtime of ED admission.
3. Consider only patients with not too much missingness.
4. Resample admissions hourly.
5. Apply Step 3 to blocked, re-sampled data.


Missing Test Functions for Admissions and vitals.
"""

# ------------------------------------ Configuration Params --------------------------------------
"""
Global and Argument variables for Vital sign processing.

TIME_VARS: list of datetime object columns.
ID_COLUMNS: identifiers of patient, admission and hospital stay
VITALS_NAMING_DIC: dictionary for renaming columns of dataframe

admission_min_count: minimum number of observations per admission
vitals_na_threshold: percentage of missing observations deemed "acceptable"
resampling_rule: frequency of averaged data to consider
admission_min_time_to_outtime: minimum length of an admission
"""
TIME_VARS = ["intime", "outtime", "next_intime", "next_outtime", "dod"]
ID_COLUMNS = ["subject_id", "hadm_id", "stay_id"]
VITALS_NAMING_DIC = {"temperature": "TEMP", "heartrate": "HR", "resprate": "RR",
                     "o2sat": "SPO2", "sbp": "SBP", "dbp": "DBP"}

admission_min_count = 3
vitals_na_threshold = 0.6
resampling_rule = "1H"
admission_min_time_to_outtime = 5


def vitals_processing():

	# --------------------- Check admission intermediate previously processed --------------------------------------
	"""Check admissions processing has been run"""

	try:
		assert os.path.exists(SAVE_FD + "admissions_intermediate.csv")

	except Exception:
		print("Current dir: ", os.getcwd())
		print("Path predicted: ", SAVE_FD + "admissions_intermediate.csv")
		raise ValueError(f"Run admissions_processing.py prior to running '{__file__}'")

	# ------------------------------------ // --------------------------------------
	"Load tables"


	"""
	Load tables, and processed admissions.

	admissions: dataframe indicating the admissions that have been processed.
	vital_signs_ed: dataframe with observation vital sign data in the ED.
	"""
	admissions = pd.read_csv(SAVE_FD + "admissions_intermediate.csv", index_col=0, header=0, parse_dates=TIME_VARS)
	vital_signs_ed = pd.read_csv(DATA_FD + "ed/vitalsign.csv", index_col=0, header=0, low_memory=False,
		                   parse_dates=["charttime"])

	# Check correct computation of admissions
	test.admissions_processed_correctly(admissions)

	# ------------------------------------- // -------------------------------------
	"""
	Process Vital Signs. Multiple steps are considered, but vital signs are re-sampled according to resampling rule,
	and then remove based on amount missingness.
	"""

	"""
	Subset to admissions pre-processed in admissions_processing.
	"""
	# Subset to admission sub-cohort and add intime/outtime information
	vitals_S1 = utils.subsetted_by(vital_signs_ed, admissions, "stay_id")
	admissions.set_index("stay_id", inplace=True)
	vitals_S1[["intime", "outtime"]] = admissions.loc[vitals_S1.stay_id.values, ["intime", "outtime"]].values
	vitals_S1.to_csv(SAVE_FD + "vitals_S1.csv", index=True, header=True)

	"""
	Subset observations within intime and outtime of ED admission. Rename columns.
	"""
	# Subset Endpoints of vital observations according to ED endpoints
	vitals_S2 = vitals_S1[vitals_S1["charttime"].between(vitals_S1["intime"], vitals_S1["outtime"])]
	vitals_S2.rename(VITALS_NAMING_DIC, axis=1, inplace=True)
	vitals_S2.to_csv(SAVE_FD + "vitals_S2.csv", index=True, header=True)

	"""
	Remove admissions with high amounts of missingness.
	"""
	# Subset to patients with enough data
	vital_feats = list(VITALS_NAMING_DIC.values())
	# vitals_S3 = utils.remove_adms_high_missingness(vitals_S2, vital_feats, "stay_id",
	# 	                                     min_count=admission_min_count, min_frac=vitals_na_threshold)
	vitals_S3 = vitals_S2
	vitals_S3.to_csv(SAVE_FD + "vitals_S3.csv", index=True, header=True)

	"""
	Compute time to end of admission, and group observations into blocks.
	"""
	# Resample admissions according to group length
	vitals_S4 = utils.compute_time_to_end(vitals_S3, id_key="stay_id", time_id="charttime", end_col="outtime")
	vitals_S4 = utils.conversion_to_block(vitals_S4, id_key="stay_id", rule=resampling_rule, time_vars=vital_feats,
		                            static_vars=["stay_id", "intime", "outtime"])
	vitals_S4.to_csv(SAVE_FD + "vitals_S4.csv", index=True, header=True)

	"""
	Apply Step 3 again with the blocked data.
	"""
	# Ensure blocks satisfy conditions - min counts, proportion of missingness AND time to final outcome
	vitals_S5 = utils.remove_adms_high_missingness(vitals_S4, vital_feats, "stay_id",
	                                     min_count=admission_min_count, min_frac=vitals_na_threshold)

	"""
	Consider those admissions with observations with at most an observations 1.5 hours before outtime 
	"""
	vitals_S5 = vitals_S5[vitals_S5["time_to_end_min"].dt.total_seconds() <= admission_min_time_to_outtime * 3600]
	vitals_S5.to_csv(SAVE_FD + "vitals_intermediate.csv", index=True, header=True)



In [ ]:
# Processing


import datetime as dt

from tqdm import tqdm



tqdm.pandas()

"""

Run -admissions_processing.py and -vitals_processing.py first.

Processing Steps:
- Subset to admissions identified previously.
- Identify windows after admission.
- Define targets as one of:
a) Death, b) ICU, c) Discharge or d) Ward.


Missing Test Functions for Admissions and vitals.
"""


def outcomes_processing():

    # ------------------------ Checking Data Loaded -------------------------------
    try:
        assert os.path.exists(SAVE_FD + "admissions_intermediate.csv")
        assert os.path.exists(SAVE_FD + "vitals_intermediate.csv")
    except Exception:
        raise ValueError(f"Run admissions_processing.py and vitals_processing.py prior to running '{__file__}'")

    # ------------------------ Configuration params --------------------------------
    """
    Global and Argument variables for Vital sign processing.
    
    TIME_VARS: datetime object variables for patient admission.
    VITALS_TIME_VARS: datetime object variables for observation data.
    """

    TIME_VARS = ["intime", "outtime", "next_intime", "next_outtime", "dod"]
    VITALS_TIME_VARS = ["intime", "outtime", "time_to_end_min", "time_to_end_max",
                        "time_to_end", f"sampled_time_to_end({resampling_rule})"]

    # ------------------------ Data Loading ------------------------------

    """
    Load data tables, including pre-processed and raw tables.
    
    admissions: processed dataframe of ED summary admission data.
    vitals: processed dataframe of ED observation data.
    transfers_core: dataframe with list of transfers within the hospital system.
    """
    admissions = pd.read_csv(SAVE_FD + "admissions_intermediate.csv", index_col=0, header=0, parse_dates=TIME_VARS)
    vitals = pd.read_csv(SAVE_FD + "vitals_intermediate.csv", index_col=0, header=0, parse_dates=VITALS_TIME_VARS)
    transfers_core = pd.read_csv(DATA_FD + "core/transfers.csv", index_col=None, header=0,
                                 parse_dates=["intime", "outtime"])
    vitals = utils.convert_to_timedelta(vitals, f"sampled_time_to_end({resampling_rule})", "time_to_end",
                                        "time_to_end_min",
                                        "time_to_end_max")

    # Check correct computation of admissions
    test.admissions_processed_correctly(admissions)
    test.vitals_processed_correctly(vitals)

    # ------------------------ Targets Processing -----------------------------

    """
    Process Target Information. Subset transfers and vitals to the relevant set of admissions
    """

    admissions_subset = utils.subsetted_by(admissions, vitals, ["stay_id"])
    transfers_subset = utils.subsetted_by(transfers_core, admissions_subset, ["subject_id", "hadm_id"])
    vitals["chartmax"] = vitals["outtime"] - vitals["time_to_end"]
    vitals["hadm_id"] = admissions.set_index("stay_id").loc[vitals.stay_id.values, "hadm_id"].values

    """
    Potential Outcomes will be regular admission to acute ward, ICU or death. We consider 4 different window sizes:
    - 4 hours, 12 hours, 24 hours and 48 hours.
    """
    time_window_1 = dt.timedelta(hours=4)
    time_window_2 = dt.timedelta(hours=12)
    time_window_3 = dt.timedelta(hours=18)
    time_window_4 = dt.timedelta(hours=24)
    time_window_5 = dt.timedelta(hours=36)
    time_window_6 = dt.timedelta(hours=48)

    # Need to include Death
    outcomes_4_hours = transfers_subset.groupby("hadm_id", as_index=True).progress_apply(
        lambda x: utils.select_death_icu_acute(x, admissions_subset, time_window_1))
    outcomes_12_hours = transfers_subset.groupby("hadm_id", as_index=True).progress_apply(
        lambda x: utils.select_death_icu_acute(x, admissions_subset, time_window_2))
    outcomes_18_hours = transfers_subset.groupby("hadm_id", as_index=True).progress_apply(
        lambda x: utils.select_death_icu_acute(x, admissions_subset, time_window_3))
    outcomes_24_hours = transfers_subset.groupby("hadm_id", as_index=True).progress_apply(
        lambda x: utils.select_death_icu_acute(x, admissions_subset, time_window_4))
    outcomes_36_hours = transfers_subset.groupby("hadm_id", as_index=True).progress_apply(
        lambda x: utils.select_death_icu_acute(x, admissions_subset, time_window_5))
    outcomes_48_hours = transfers_subset.groupby("hadm_id", as_index=True).progress_apply(
        lambda x: utils.select_death_icu_acute(x, admissions_subset, time_window_6))

    # Ensure all patients have only one class
    assert outcomes_4_hours.iloc[:, :-1].sum(axis=1).eq(1).all()
    assert outcomes_12_hours.iloc[:, :-1].sum(axis=1).eq(1).all()
    assert outcomes_18_hours.iloc[:, :-1].sum(axis=1).eq(1).all()
    assert outcomes_24_hours.iloc[:, :-1].sum(axis=1).eq(1).all()
    assert outcomes_36_hours.iloc[:, :-1].sum(axis=1).eq(1).all()
    assert outcomes_48_hours.iloc[:, :-1].sum(axis=1).eq(1).all()

    """
    Final processing, ensure admissions and observations match with patient ids
    """
    # Subset vitals and admission data
    admissions_keep = outcomes_4_hours.index.tolist()
    admissions_final = admissions_subset[admissions_subset.hadm_id.isin(admissions_keep)]
    vitals_final = vitals[vitals.hadm_id.isin(admissions_keep)]

    """
    Add static variables to input data.
    """
    static_vars = ["gender", "age", "ESI"]
    vitals_final[static_vars] = admissions_final.set_index("hadm_id").loc[
        vitals_final.hadm_id.values, static_vars].values
    vitals_final["gender"] = vitals_final.loc[:, "gender"].replace(to_replace=["M", "F"], value=[1, 0])
    vitals_final["charttime"] = vitals_final.loc[:, "outtime"].values - vitals_final.loc[:,
                                                                        "sampled_time_to_end(1H)"].values

    """
    Save Data and Print Basic Information.
    """

    # Number of Patients and number of observations.
    print(f"Number of cohort patient: {vitals_final.stay_id.nunique()}")
    print(f"Number of observations: {vitals_final.shape[0]}")

    print(f"Sample outcome distribution: {outcomes_4_hours.sum(axis=0)}")

    # Save to output variables
    process_fd = DATA_FD + "processed/"

    if not os.path.exists(process_fd):
        os.makedirs(process_fd)

    # Save general
    vitals_final.to_csv(SAVE_FD + "vitals_final.csv", index=True, header=True)
    admissions_final.to_csv(SAVE_FD + "admissions_final.csv", index=True, header=True)

    # Save for input
    vitals_final.to_csv(process_fd + "vitals_process.csv", index=True, header=True)
    admissions_final.to_csv(process_fd + "admissions_process.csv", index=True, header=True)
    outcomes_4_hours.to_csv(process_fd + "outcomes_4h_process.csv", index=True, header=True)
    outcomes_12_hours.to_csv(process_fd + "outcomes_12h_process.csv", index=True, header=True)
    outcomes_18_hours.to_csv(process_fd + "outcomes_18h_process.csv", index=True, header=True)
    outcomes_24_hours.to_csv(process_fd + "outcomes_24h_process.csv", index=True, header=True)
    outcomes_36_hours.to_csv(process_fd + "outcomes_36h_process.csv", index=True, header=True)
    outcomes_48_hours.to_csv(process_fd + "outcomes_48h_process.csv", index=True, header=True)


In [ ]:
# Run admissions if not processed
    if not os.path.exists("data/MIMIC/interim/admissions_intermediate.csv"):
        admissions_processing()

    # Run vitals
    if not os.path.exists("data/MIMIC/interim/vitals_intermediate.csv"):
        vitals_processing()

    # Run outcomes
    if not os.path.exists("data/MIMIC/interim/vitals_final.csv"):
        outcomes_processing()

    pass

SVM Model definition

In [ ]:
"""
Define Model Class for SVMAll model. SVM is fit to concatenated trajectories.
"""

import json

from sklearn.svm import SVC
import numpy as np



SVMALL_INPUT_PARAMS = ["C", "kernel", "degree", "gamma", "coef0", "shrinking", "probability", "tol", "class_weight",
                       "random_state", "verbose"]


class SVMAll(SVC):
    """
    Model Class Wrapper for an SVM model training on all (time, feature) pair values.
    """

    def __init__(self, data_info: dict = {}, probability: bool = True, **kwargs):
        """
        Initialise object with model configuration.

        Params:
        - data_info: dict, dictionary containing dataset information, including objects and properties.
        - kwargs: model configuration parameters
        """

        # Get proper model_config
        self.model_config = {key: value for key, value in kwargs.items() if key in SVMALL_INPUT_PARAMS}

        if "probability" not in self.model_config.keys():
            self.model_config["probability"] = probability

        # Initialise other useful information
        self.run_num = 1
        self.model_name = "SVMALL"

        # Useful for consistency
        self.training_params = {}

        # Initialise SVM object with this particular model config
        super().__init__(**self.model_config, verbose=True)

    def train(self, data_info, **kwargs):
        """
        Wrapper method for fitting the model to input data.

        Params:
        - probability: bool value, indicating whether model should output hard outcome assignments, or probabilistic.
        - data_info: dictionary with data information, objects and parameters.
        """

        # Unpack relevant data information
        X_train, X_val, X_test = data_info["X"]
        y_train, y_val, y_test = data_info["y"]
        data_name = data_info["data_load_config"]["data_name"]

        # Update run_num to make space for new experiment
        run_num = self.run_num
        save_fd = f"experiments/{data_name}/{self.model_name}/"

        while os.path.exists(save_fd + f"run{run_num}/"):
            run_num += 1

        # make new folder and update run num
        os.makedirs(save_fd + f"run{run_num}/")
        self.run_num = run_num

        # Fit to concatenated X_train, X_val
        X_train = np.concatenate((X_train, X_val), axis=0)
        y_train = np.concatenate((y_train, y_val), axis=0)

        # Get shape and flatten array
        N_test, T, D_f = X_train.shape
        X = X_train.reshape(-1, X_train.shape[-1])
        y_per_feat = np.repeat(y_train.reshape(-1, 1, 4), repeats=T, axis=1)
        y = np.argmax(y_per_feat, axis=-1).reshape(-1)

        # Fit model
        self.fit(X, y, sample_weight=None)

        return None

    def analyse(self, data_info, **kwargs):
        """
        Evaluation method to compute and save output results.

        Params:
        - data_info: dictionary with data information, objects and parameters.

        Returns:
            - y_pred: dataframe of shape (N, output_dim) with outcome probability prediction.
            - outc_pred: Series of shape (N, ) with predicted outcome based on most likely outcome prediction.
            - y_true: dataframe of shape (N, output_dim) ith one-hot encoded true outcome.

        Saves a variety of model information, as well.
        """

        # Unpack test data
        _, _, X_test = data_info["X"]
        _, _, y_test = data_info["y"]

        # Get basic data information
        data_properties = data_info["data_properties"]
        outc_dims = data_properties["outc_names"]
        data_load_config = data_info["data_load_config"]
        data_name = data_load_config["data_name"]

        # Obtain the ids for patients in test set
        id_info = data_info["ids"][-1]
        pat_ids = id_info[:, 0, 0]

        # Define save_fd, track_fd
        save_fd = f"results/{data_name}/{self.model_name}/run{self.run_num}/"

        if not os.path.exists(save_fd):
            os.makedirs(save_fd)

        # Make prediction on test data
        if self.model_config["probability"] is True:
            X_test = X_test.reshape(-1, X_test.shape[-1])
            output_test = self.predict_proba(X_test).reshape(pat_ids.size, -1, 4)
            output_test = np.mean(output_test, axis=1)

        else:
            # Predict gives categorical vector, and we one-hot encode output.
            output_test = np.eye(y_test.shape[-1])[self.predict(X_test)]

        # First, compute predicted y estimates
        y_pred = pd.DataFrame(output_test, index=pat_ids, columns=outc_dims)
        outc_pred = pd.Series(np.argmax(output_test, axis=-1), index=pat_ids)
        y_true = pd.DataFrame(y_test, index=pat_ids, columns=outc_dims)


        # Define clusters as outcome predicted groups
        pis_pred = y_pred
        clus_pred = outc_pred

        # Get model config
        model_config = self.model_config

        # ----------------------------- Save Output Data --------------------------------
        # Useful objects
        y_pred.to_csv(save_fd + "y_pred.csv", index=True, header=True)
        outc_pred.to_csv(save_fd + "outc_pred.csv", index=True, header=True)
        clus_pred.to_csv(save_fd + "clus_pred.csv", index=True, header=True)
        pis_pred.to_csv(save_fd + "pis_pred.csv", index=True, header=True)
        y_true.to_csv(save_fd + "y_true.csv", index=True, header=True)

        # save model parameters
        with open(save_fd + "data_config.json", "w+") as f:
            json.dump(data_info["data_load_config"], f, indent=4)

        with open(save_fd + "model_config.json", "w+") as f:
            json.dump(model_config, f, indent=4)

        # Return objects
        outputs_dic = {"save_fd": save_fd, "model_config": self.model_config,
                       "y_pred": y_pred, "class_pred": outc_pred, "clus_pred": clus_pred, "pis_pred": pis_pred,
                       "y_true": y_true
                       }

        # Print Data
        print(f"\n\n Results Saved under {save_fd}")

        return outputs_dic


Camelot Model definition

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
<<<<<<< HEAD

# ----------------------------------------------------------------------------------
"Imports"

import tensorflow as tf
from tensorflow.math import squared_difference, multiply, divide
=======
Loss, Metrics and Callback functions to use for model


"""
import os
import numpy as np
import pandas as pd

import tensorflow as tf
>>>>>>> develop
import tensorflow.keras.callbacks as cbck

from sklearn.metrics import roc_auc_score as roc
from sklearn.metrics.cluster import contingency_matrix
from sklearn.metrics import adjusted_rand_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.metrics import normalized_mutual_info_score, silhouette_score

# ----------------------------------------------------------------------------------
"Utility Functions and Global Params"

<<<<<<< HEAD
=======
LOGS_DIR = "experiments/camelot/"

>>>>>>> develop

def log(tensor):
    return tf.cast(tf.math.log(tensor + 1e-8), dtype="float32")


<<<<<<< HEAD
=======
def np_log(array):
    return np.log(array + 1e-8)


>>>>>>> develop
def purity_score(y_true, y_pred):
    # compute confusion matrix
    contingency_matrix_ = contingency_matrix(y_true, y_pred)

    return np.sum(np.amax(contingency_matrix_, axis=0)) / np.sum(contingency_matrix_)


# ------------------------------------------------------------------------------------
"""Loss Functions"""


def l_pred(y_true, y_pred, weights=None, name='pred_clus_loss'):
    """
<<<<<<< HEAD
    Predictive clustering loss

    Params:
    - y_true: tensor of shape (bs, num_outcs)
    - y_pred: tensor of shape (bs, num_outcs)
    """

    # Check for whether weights are given or not
    if weights is None:
        weights = tf.cast(tf.constant(np.ones(shape=y_true.shape) / y_true.shape[-1]), dtype=np.float32)
    else:
        weights = tf.convert_to_tensor(value=weights, dtype="float32")

    # Compute batch and return
    batch_loss = - tf.reduce_mean(tf.reduce_sum(weights * y_true * log(y_pred), axis=-1), name=name)

    return batch_loss


def l_clus(clusters, name='emb_sep_L'):
    """
    Cluster representation separation loss.

    Params:
    - clusters: tensor of shape (K, _ )
    """

    # Expand dims to take advantage of broadcasting
    embedding_column = tf.expand_dims(clusters, axis=1)          # shape (K, 1, _)
    embedding_row = tf.expand_dims(clusters, axis=0)             # shape (1, K, _)

    # Compute L1 distance
    pairwise_loss = - tf.reduce_sum((embedding_column - embedding_row) ** 2, axis=-1)  # shape K, K
    loss = tf.reduce_sum(pairwise_loss, axis=None, name=name)

    # normalise by K(K-1=/2
    K = clusters.get_shape()[0]
    norm_factor = K * (K - 1) / 2
    norm_loss = tf.math.divide(loss, tf.cast(norm_factor, dtype="float32"))
=======
    Negative weighted predictive clustering loss. Computes Cross-entropy between categorical y_true and y_pred.
    This is minimised when y_pred matches y_true.

    Params:
    - y_true: array-like of shape (bs, num_outcs) of one-hot encoded true class.
    - y_pred: array-like of shape (bs, num_outcs) of probability class predictions.
    - weights: array-like of shape (num_outcs) of weights to multiply cross-entropy terms. (default None).
    - name: name to give to operation.

    If weights is None, defaults to regular cross-entropy calculation.

    Returns:
    - loss_value: score indicating corresponding loss.
    """

    # If weights is None, return weights as set of 1s.
    if weights is None:
        weights = tf.cast(tf.constant(np.ones(shape=y_true.shape) / y_true.shape[-1]), dtype=np.float32)

    # Compute batch
    batch_neg_ce = tf.reduce_sum(weights * y_true * log(y_pred), axis=-1)

    # Average over batch
    loss_value = tf.reduce_mean(batch_neg_ce, name=name)

    return loss_value


def l_clus(cluster_reps, name='embedding_sep_loss'):
    """Compute Embedding separation Loss on embedding vectors."""
    """Cluster representation separation loss. Computes negative euclidean distance summed over pairs of cluster 
    representation vectors. This loss is minimised as cluster vectors are separated 

    Params:
    - cluster_reps: array-like of shape (K, latent_dim) of cluster representation vectors.
    - name: name to give to operation.

    Returns:
    - norm_loss: score indicating corresponding loss.
    """

    # Expand input to allow broadcasting
    embedding_column = tf.expand_dims(cluster_reps, axis=1)  # shape (K, 1, latent_dim)
    embedding_row = tf.expand_dims(cluster_reps, axis=0)  # shape (1, K, latent_dim)

    # Compute pairwise Euclidean distance between cluster vectors, and sum over pairs of clusters.
    pairwise_loss = tf.reduce_sum((embedding_column - embedding_row) ** 2, axis=-1)
    loss = - tf.reduce_sum(pairwise_loss, axis=None, name=name)

    # normalise by factor
    K = cluster_reps.get_shape()[0]
    norm_loss = loss / (K * (K - 1))
>>>>>>> develop

    return norm_loss


<<<<<<< HEAD
def l_dist(clus_prob, name = "clus_sel_loss"):
    """
    Cluster selection loss.

    Params:
    - clus_prob: tensor of shape (batch_size, num_clusters).
    """

    # Compute average distribution over each cluster
    avg_prob_per_clus = tf.reduce_mean(clus_prob, axis=-1, name=name)

    # Compute negative entropy - minimised for uniform distribution.
    neg_entropy = tf.reduce_sum(multiply(avg_prob_per_clus, log(avg_prob_per_clus)))
=======
def l_dist(clusters_prob):
    """
    Cluster distribution loss. Computes negative entropy of cluster distribution probability values.
    This is minimised when the cluster distribution is uniform (all clusters similar size).

    Params:
    - clusters_prob: array-like of shape (bs, K) of cluster_assignments distributions.
    - name: name to give to operation.

    Returns:
    - loss_value: score indicating corresponding loss.
    """

    # Calculate average cluster assignment distribution
    clus_avg_prob = tf.reduce_mean(clusters_prob, axis=0)

    # Compute negative entropy
    neg_entropy = tf.reduce_sum(clus_avg_prob * log(clus_avg_prob))
>>>>>>> develop

    return neg_entropy


# ----------------------------------------------------------------------------------
<<<<<<< HEAD
"Useful information to print during training."


class y_clus_cross_entropy(cbck.Callback):
    """
    Compute normalised Cross-Entropy Loss between cluster phenotypes.
    Smaller values represent more separation of y_clusters.
    """

    def __init__(self, validation_data: tuple = (), interval: int = 5):
        super().__init__()
        self.interval = interval
        self.X_val, _ = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            ce_sep, epsilon = 0, 1e-9
            K = self.model.embeddings.numpy().shape[0]

            # Compute embedding phenotypes
            clus_phenotypes = self.model.Predictor(self.model.embeddings).numpy() + epsilon
=======
"Callback methods to update training procedure."


class CEClusSeparation(cbck.Callback):
    """
    Callback method to print Normalised Cross-Entropy separation between cluster phenotypes.
    Higher values indicate higher separation (which is good!)

    Params:
    - validation_data: tuple of X_val, y_val data
    - weighted: bool, indicating whether outcomes should be weighted. (default = False)
    - interval: interval between epochs on which to print values. (default = 5)
    """

    def __init__(self, validation_data: tuple, weighted: bool = False, interval: int = 5):
        super().__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data

        # Compute outcome weights if weighted=True
        if weighted is True:
            class_numbers = tf.reduce_sum(self.y_val, axis=0).numpy()
            self.weights = class_numbers / np.sum(class_numbers)

        else:
            self.weights = np.ones(shape=(self.y_val.get_shape()[0]))

    def on_epoch_end(self, epoch, logs=None, **kwargs):

        # Print information if matches interval epoch length
        if epoch % self.interval == 0:

            # Initialise callback value, and determine K
            cbck_value, K = 0, self.model.cluster_reps.numpy().shape[0]
            clus_phenotypes = self.model.Predictor(self.model.cluster_reps).numpy()
>>>>>>> develop

            # Iterate over all pairs of clusters and compute symmetric CE
            for i in range(K):
                for j in range(i + 1, K):
<<<<<<< HEAD
                    ce_sep += - np.sum(clus_phenotypes[i, :] * np.log(clus_phenotypes[j, :]))
                    ce_sep += - np.sum(clus_phenotypes[j, :] * np.log(clus_phenotypes[i, :]))

            # normalise and print output
            norm_loss = ce_sep / (K * (K + 1))
            print("End of Epoch {:d} - CE sep : {:.4f}".format(epoch, norm_loss))


class ConfusionMatrix(cbck.Callback):
    """Display Confusion Matrix of predicted outcomes vs target outcomes."""

    def __init__(self, validation_data=(), interval=5):
        super().__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data
        self.C = self.y_val.shape[-1]  # num_classes

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            cm_output = np.zeros(shape=(self.C, self.C))

            # Compute prediction and true values in categorical format.
            model_output = (self.model(self.X_val)).numpy()
            y_pred = np.argmax(model_output, axis=-1)
            y_true = np.argmax(self.y_val, axis=-1)

            # Iterate through classes
            for true_class in range(self.C):
                for pred_class in range(self.C):
                    num_samples = np.logical_and(y_pred == pred_class, y_true == true_class).sum()
                    cm_output[true_class, pred_class] = num_samples

            print("End of Epoch {:d} - Confusion matrix: \n {}".format(epoch, cm_output.astype(int)))


class AUROC(cbck.Callback):
    """Compute AUROC on Validation Data."""

    def __init__(self, validation_data=(), interval=5):
=======
                    cbck_value += - np.sum(clus_phenotypes[i, :] * np_log(clus_phenotypes[j, :]))
                    cbck_value += - np.sum(clus_phenotypes[j, :] * np_log(clus_phenotypes[i, :]))

            # normalise and print output
            cbck_value = cbck_value / (K * (K + 1))

            print("End of Epoch {:d} - CE sep : {:.4f}".format(epoch, cbck_value))


class ConfusionMatrix(cbck.Callback):
    """
    Callback method to print Confusion Matrix over data.

    Output is a matrix indicating the amount of patients assigned to a target class and with a certain true class.

    Params:
    - validation_data: tuple of X_val, y_val data
    - interval: interval between epochs on which to print values. (default = 5)
    """

    def __init__(self, validation_data: tuple, interval: int = 5):
        super().__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data

        # Compute number of outcomes
        self.num_outcs = self.y_val.shape[-1]

    def on_epoch_end(self, epoch, logs=None):

        # Print information if matches interval epoch length
        if epoch % self.interval == 0:

            # Initialise output Confusion matrix
            cm_output = np.zeros(shape=(self.num_outcs, self.num_outcs))

            # Compute prediction and true values in categorical format.
            y_pred = (self.model(self.X_val)).numpy()
            class_pred = np.argmax(y_pred, axis=-1)
            class_true = np.argmax(self.y_val, axis=-1)

            # Iterate through classes
            for true_class in range(self.num_outcs):
                for pred_class in range(self.num_outcs):
                    num_samples = np.logical_and(class_pred == pred_class, class_true == true_class).sum()
                    cm_output[true_class, pred_class] = num_samples

            # Print as pd.dataframe
            index = [f"TC{class_}" for class_ in range(1, self.num_outcs + 1)]
            columns = index

            cm = pd.DataFrame(cm_output, index=index, columns=columns)

            print("End of Epoch {:d} - Confusion matrix: \n {}".format(epoch, cm.astype(int)))


class AUROC(cbck.Callback):
    """
    Callback method to display AUROC value for predicted y.

    Params:
    - validation_data: tuple of X_val, y_val data
    - interval: interval between epochs on which to print values. (default = 5)
    """

    def __init__(self, validation_data: tuple, interval: int = 5):
>>>>>>> develop
        super().__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data

<<<<<<< HEAD
    def on_epoch_end(self, epoch, logs={}):
=======
    def on_epoch_end(self, epoch, logs=None):
>>>>>>> develop
        if epoch % self.interval == 0:
            # Compute predictions
            y_pred = self.model(self.X_val).numpy()

            # Compute ROC
            roc_auc_score = roc(y_true=self.y_val, y_score=y_pred,
                                average=None, multi_class='ovr')

<<<<<<< HEAD
            print("End of Epoch {:d} - ROC score: {}".format(epoch, roc_auc_score))


class PrintClustersUsed(cbck.Callback):
    """Print Number of Clusters and Cluster Distribution with samples assigned to them."""

    def __init__(self, validation_data=(), interval=5):
=======
            print("End of Epoch {:d} - OVR ROC score: {}".format(epoch, roc_auc_score))


class PrintClusterInfo(cbck.Callback):
    """
    Callback method to display cluster distribution information assignment.

    Params:
    - validation_data: tuple of X_val, y_val data
    - interval: interval between epochs on which to print values. (default = 5)
    """

    def __init__(self, validation_data: tuple, interval: int = 5):
>>>>>>> develop
        super().__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data

<<<<<<< HEAD
    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            # Compute cluster assignment
            clus_pred = self.model.Identifier(self.model.Encoder(self.X_val)).numpy()
            num_clusters = np.unique(np.argmax(clus_pred, axis=-1))
            avg_cluster_dist = np.mean(clus_pred, axis=-1)

            # Print Information
            print("End of Epoch {:d} - num_clusters {} - cluster dist {}".format(epoch, num_clusters, avg_cluster_dist))


class SupervisedTargetMetrics(cbck.Callback):
    """Print Scores for all given Supervised Target Metrics (NMI, ARS, Purity) on validation data during training."""

    def __init__(self, validation_data=(), interval=5):
=======
    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.interval == 0:

            # Compute cluster_predictions
            clus_pred = self.model.Identifier(self.model.Encoder(self.X_val)).numpy()
            clus_assign = np.argmax(clus_pred, axis=-1)
            K = clus_pred.shape[-1]

            # Compute "hard" cluster assignment numbers
            hard_cluster_num = np.zeros(shape=K)
            for clus_id in range(self.K):
                hard_cluster_num[clus_id] = np.sum(clus_assign == clus_id)
            hard_cluster_num = pd.Series(hard_cluster_num, index=[f"Clus {k}" for k in range(1, K + 1)])

            # Compute average cluster distribution
            avg_cluster_dist = np.mean(clus_pred, axis=0)

            # Print Information
            print("End of Epoch {:d} - hard_cluster_info {} - avg cluster prob dist {}".format(epoch, hard_cluster_num,
                                                                                               avg_cluster_dist))


class SupervisedTargetMetrics(cbck.Callback):
    """
    Callback method to display supervised target metrics: Normalised Mutual Information, Adjusted Rand Score and
    Purity Score

    Params:
    - validation_data: tuple of X_val, y_val data
    - interval: interval between epochs on which to print values. (default = 5)
    """

    def __init__(self, validation_data: tuple, interval: int = 5):
>>>>>>> develop
        super().__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data

<<<<<<< HEAD
    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            # Compute y_pred, y_true in categorical format.
            model_output = (self.model(self.X_val)).numpy()
            y_pred = np.argmax(model_output, axis=-1)
            y_true = np.argmax(self.y_val, axis=-1).reshape(-1)

            # Target metrics
            nmi = normalized_mutual_info_score(labels_true=y_true, labels_pred=y_pred)
            ars = adjusted_rand_score(labels_true=y_true, labels_pred=y_pred)
            purity = purity_score(y_true=y_true, y_pred=y_pred)
=======
    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.interval == 0:
            # Compute y_pred, y_true in categorical format.
            model_output = (self.model(self.X_val)).numpy()
            class_pred = np.argmax(model_output, axis=-1)
            class_true = np.argmax(self.y_val, axis=-1).reshape(-1)

            # Target metrics
            nmi = normalized_mutual_info_score(labels_true=class_true, labels_pred=class_pred)
            ars = adjusted_rand_score(labels_true=class_true, labels_pred=class_pred)
            purity = purity_score(y_true=class_true, y_pred=class_pred)
>>>>>>> develop

            print("End of Epoch {:d} - NMI {:.2f} , ARS {:.2f} , Purity {:.2f}".format(epoch, nmi, ars, purity))


class UnsupervisedTargetMetrics(cbck.Callback):
<<<<<<< HEAD
    """Print Scores for all Unsupervised metrics (DBS, CHS, SIL) on validation data (inc. latent) over training."""

    def __init__(self, validation_data=(), interval=5):
        super().__init__()
        self.interval = interval
        self.latent_reps = None
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            # Compute predictions and latent representations
            self.latent_reps = self.model.Encoder(self.X_val)
            model_output = (self.model(self.X_val)).numpy()
            y_pred = np.argmax(model_output, axis=-1)
=======
    """
    Callback method to display unsupervised target metrics: Davies-Bouldin Score, Calinski-Harabasz Score,
    Silhouette Score

    Params:
    - validation_data: tuple of X_val, y_val data
    - interval: interval between epochs on which to print values. (default = 5)
    """

    def __init__(self, validation_data: tuple, interval: int = 5):
        super().__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.interval == 0:
            # Compute predictions and latent representations
            latent_reps = self.model.Encoder(self.X_val)
            model_output = (self.model(self.X_val)).numpy()
            clus_pred = np.argmax(model_output, axis=-1)
>>>>>>> develop

            # Reshape input data and allow feature comparison
            X_val_2d = np.reshape(self.X_val, (self.X_val[0], -1))

            # Compute metrics
<<<<<<< HEAD
            dbs = davies_bouldin_score(X_val_2d, labels=y_pred)
            dbs_l = davies_bouldin_score(self.latent_reps, labels=y_pred)
            chs = calinski_harabasz_score(X_val_2d, labels=y_pred)
            chs_l = calinski_harabasz_score(self.latent_reps, labels=y_pred)
            sil = silhouette_score(X=X_val_2d, labels=y_pred, random_state=self.model.seed)
            sil_l = silhouette_score(X=self.latent_reps, labels=y_pred, random_state=self.model.seed)

            print(f"""End of Epoch {epoch:d} (score, score on latent): 
=======
            dbs = davies_bouldin_score(X_val_2d, labels=clus_pred)
            dbs_l = davies_bouldin_score(latent_reps, labels=clus_pred)
            chs = calinski_harabasz_score(X_val_2d, labels=clus_pred)
            chs_l = calinski_harabasz_score(latent_reps, labels=clus_pred)
            sil = silhouette_score(X=X_val_2d, labels=clus_pred, random_state=self.model.seed)
            sil_l = silhouette_score(X=latent_reps, labels=clus_pred, random_state=self.model.seed)

            print(f"""End of Epoch {epoch:d} (score, latent score): 
>>>>>>> develop
                        DBS {dbs:.2f}, {dbs_l:.2f}   
                        CHS {chs:.2f}, {chs_l:.2f}  
                        SIL {sil:.2f}, {sil_l:.2f}""")


<<<<<<< HEAD
def compute_metric(metric_name):
    """Given metric shorthand, return corresponding callback."""
    if "auc" == metric_name.lower() or "roc" == metric_name.lower():
        return AUROC

    elif "ce_sep" == metric_name.lower():
        return CESeparation

    elif "conf_matrix" == metric_name.lower():
        return ConfusionMatrix

    elif "clus_dist" == metric_name.lower():
        return PrintClustersUsed

    elif "sup_targets" == metric_name.lower():
        return SupervisedTargetMetrics

    elif "unsup_targets" == metric_name.lower():
        return UnsupervisedTargetMetrics


def get_callbacks(track_loss, early_stop=True, lr_scheduler=True, tensorboard=True, min_delta=0.0001, patience=100):
    """Generate list of callbacks, given input params.

    Params:
        - track_loss: str, name of main loss to keep track of.
=======
def cbck_list(summary_name: str):
    """
    Shorthand for callbacks above.

    Params:
    - summary_name: str containing shorthands for different callbacks.
    """
    extra_callback_list = set([])

    if "auc" in summary_name.lower() or "roc" in summary_name.lower():
        extra_callback_list.update([AUROC])

    if "clus_sep" in summary_name.lower() or "clus_phen" in summary_name.lower():
        extra_callback_list.update([CEClusSeparation])

    if "cm" in summary_name.lower() or "conf_matrix" in summary_name.lower():
        extra_callback_list.update([ConfusionMatrix])

    if "clus_info" in summary_name.lower():
        extra_callback_list.update([PrintClusterInfo])

    if "sup_scores" in summary_name.lower():
        extra_callback_list.update([SupervisedTargetMetrics])

    if "unsup_scores" in summary_name.lower():
        extra_callback_list.update([UnsupervisedTargetMetrics])

    return list(extra_callback_list)


def get_callbacks(track_loss: str, other_cbcks: str = "", early_stop: bool = True, lr_scheduler: bool = True,
                  tensorboard: bool = True,
                  min_delta: float = 0.0001, patience: int = 100):
    """
    Generate complete list of callbacks given input configuration.

    Params:
        - track_loss: str, name of main loss to keep track of.
        - other_cbcks: str, list of other callbacks to consider (default = "", which selects None).
>>>>>>> develop
        - early_stop: whether to stop training early in case of no progress. (default = True)
        - lr_scheduler: dynamically update learning rate. (default = True)
        - tensorboard: write tensorboard friendly logs which can then be visualised. (default = True)
        - min_delta: if early stopping, the interval on which to check improvement or not.
        - patience: how many epochs to wait until checking for improvements.
        """
<<<<<<< HEAD
    cbck_list = []

    # Handle saving paths and folders
    logs_dir = "experiments/main/"
=======
    callbacks = set([])

    # Handle saving paths and folders
    logs_dir = LOGS_DIR
>>>>>>> develop
    if not os.path.exists(logs_dir):
        os.makedirs(logs_dir)

    # Save Folder is first run that has not been previously computed
    run_num = 1
    while os.path.exists(logs_dir + f"run{run_num}/"):
        run_num += 1
<<<<<<< HEAD
    save_fd = logs_dir + f"run{run_num}/"
    assert not os.path.exists(save_fd)
    os.makedirs(save_fd)
    os.makedirs(save_fd + "logs/")

    # Model Weight saving callback
    checkpoint = cbck.ModelCheckpoint(filepath=save_fd + "models/checkpoints/epoch-{epoch}", save_best_only=True,
                                      monitor=track_loss, save_freq="epoch")
    csv_logger = cbck.CSVLogger(filename=save_fd + "loss_tracker.csv", separator=",", append=True)
    cbck_list.append(checkpoint)
    cbck_list.append(csv_logger)

    if early_stop:
        cbck_list.append(cbck.EarlyStopping(monitor='val_' + track_loss, mode="min", restore_best_weights=True,
                                            min_delta=min_delta, patience=patience))

    if lr_scheduler:
        cbck_list.append(cbck.ReduceLROnPlateau(monitor='val_' + track_loss, mode='min', cooldown=15,
                                                min_lr=0.00001, factor=0.25))

    if tensorboard:
        cbck_list.append(cbck.TensorBoard(log_dir=save_fd + "logs/", histogram_freq=1))

    return cbck_list, run_num
=======

    # Save as new run
    save_fd = logs_dir + f"run{run_num}/"
    assert not os.path.exists(save_fd)

    os.makedirs(save_fd)
    os.makedirs(save_fd + "logs/")

    # ------------------ Start Loading callbacks ---------------------------

    # Load custom callbacks first
    callbacks.update(cbck_list(other_cbcks))

    # Model Weight saving callback
    checkpoint = cbck.ModelCheckpoint(filepath=save_fd + "models/checkpoints/epoch-{epoch}", save_best_only=True,
                                      monitor=track_loss, save_freq="epoch")
    callbacks.update([checkpoint])

    # Logging Loss values
    csv_logger = cbck.CSVLogger(filename=save_fd + "training/loss_tracker.csv", separator=",", append=True)
    callbacks.update([csv_logger])

    # Check if Early stoppage is added
    if early_stop:
        callbacks.update([cbck.EarlyStopping(monitor='val_' + track_loss, mode="min", restore_best_weights=True,
                                             min_delta=min_delta, patience=patience)])

    # Check if LR Scheduling is in place
    if lr_scheduler:
        callbacks.update([cbck.ReduceLROnPlateau(monitor='val_' + track_loss, mode='min', cooldown=15,
                                                 min_lr=0.00001, factor=0.25)])

    # Check if Tensorboard is active
    if tensorboard:
        callbacks.update([cbck.TensorBoard(log_dir=save_fd + "logs/", histogram_freq=1)])

    return callbacks, run_num
>>>>>>> develop


In [ ]:


# ------------ Import Libraries ---------------
import tensorflow as tf
from tensorflow import linalg
from tensorflow.keras.layers import Dense, Dropout, Layer, LSTM
from tensorflow.keras.regularizers import l1_l2 as mix_l1_l2_reg


# ------------ Utility Functions --------------
<<<<<<< HEAD
def _norm_abs(inputs, axis=1):
    """
    Given input data, compute normalisation according to L1 metric for given axis 1.
    """
    abs_input = tf.math.abs(inputs)  # Compute vector of absolute values.

    return tf.math.divide(abs_input, tf.reduce_sum(abs_input, axis=axis, keepdims=True))


def _estimate_alpha(feature_reps, targets):
    """
    alpha parameters OLS estimation given projected input features and targets.
    feature_reps: shape (bs, T, d, units)
    targets: shape (bs, T, units)
=======
def _estimate_alpha(feature_reps, targets):
    """
    alpha parameters OLS estimation given projected input features and targets.

    Params:
    - feature_reps: array-like of shape (bs, T, d, units)
    - targets: array-like of shape (bs, T, units)

    returns:
    - un-normalised alpha weights: array-like of shape (bs, T, d)
>>>>>>> develop
    """
    X_T, X = feature_reps, linalg.matrix_transpose(feature_reps)

    # Compute matrix inversion
    X_TX_inv = linalg.inv(linalg.matmul(X_T, X))
    X_Ty = linalg.matmul(X_T, tf.expand_dims(targets, axis=-1))

    # Compute likely scores
    alpha_hat = linalg.matmul(X_TX_inv, X_Ty)

<<<<<<< HEAD
    return tf.squeeze(alpha_hat)  # shape (bs, T, d) (NOT normalised)
=======
    return tf.squeeze(alpha_hat)
>>>>>>> develop


def _estimate_gamma(o_hat, cluster_targets):
    """
    Estimate gamma parameters through OLS estimation given projected input features and targets.
<<<<<<< HEAD
    o_hat: shape (bs, T, units)
    targets: shape (K, units)
=======

    Params:
    - o_hat: array-like of shape (bs, T, units)
    - targets: array-like of shape (K, units)

    returns:
    - gamma_weights: array-like of shape (bs, K, T)
>>>>>>> develop
    """
    X_T = tf.expand_dims(o_hat, axis=1)
    X = linalg.matrix_transpose(X_T)
    y = tf.expand_dims(tf.expand_dims(cluster_targets, axis=0), axis=-1)

    # Compute inversion
    X_TX_inv = linalg.inv(linalg.matmul(X_T, X))
    X_Ty = linalg.matmul(X_T, y)

    # Compute gamma
    gamma_hat = linalg.matmul(X_TX_inv, X_Ty)

    return tf.squeeze(gamma_hat)


<<<<<<< HEAD
=======
def _norm_abs(array, axis: int = 1):
    """
    Compute L1 normalisation of array according to axis.

    Params:
    - array: array-like object.
    - axis: integer.

    returns:
    - normalised array according to axis.
    """
    array_abs = tf.math.abs(array)

    output = array_abs / tf.reduce_sum(array_abs, axis=axis, keepdims=True)

    return output


>>>>>>> develop
# ------------ MLP definition ---------------
class MLP(Layer):
    """
    Multi-layer perceptron (MLP) neural network architecture.

    Params:
    - output_dim : int, dimensionality of output space for each sub-sequence.
    - hidden_layers : int, Number of "hidden" feedforward layers. (default = 2)
<<<<<<< HEAD
    - hidden_nodes : int, For "hidden" feedforward layers, the dimensionality  of the output space. (default = 30)
    - activation_fn : str/fn, The activation function to use. (default = 'sigmoid')
    - output_fn : str/fn, The activation function on the output of the MLP. (default = 'softmax').
    - dropout : float, dropout rate (default = 0.6).
    - reg_params : tuple of floats for regularization (default = (0.01, 0.01))
    - seed : int, Seed used for random mechanisms (default = 4347)
    - name : str, name on which to save layer. (default = 'decoder')
    """

    def __init__(self, output_dim: int, hidden_layers: int = 2, hidden_nodes: int = 30, activation_fn='sigmoid',
                 output_fn='softmax', dropout: float = 0.6, reg_params=(0.01, 0.01), seed: int = 4347,
                 name: float = 'MLP'):
=======
    - hidden_nodes : int, For "hidden" feedforward layers, the dimensionality of the output space. (default = 30)
    - activation_fn : str/fn, The activation function to use. (default = 'sigmoid')
    - output_fn : str/fn, The activation function on the output of the MLP. (default = 'softmax').
    - dropout : float, dropout rate (default = 0.6).
    - regulariser_params : tuple of floats for regularization (default = (0.01, 0.01))
    - seed : int, Seed used for random mechanisms (default = 4347)
    - name : str, name on which to save layer. (default = 'MLP')
    """

    def __init__(self, output_dim: int, hidden_layers: int = 2, hidden_nodes: int = 30, activation_fn='sigmoid',
                 output_fn='softmax', dropout: float = 0.6, regulariser_params: tuple = (0.01, 0.01), seed: int = 4347,
                 name: str = 'MLP'):
>>>>>>> develop

        # Block parameters
        super().__init__(name=name)
        self.output_dim = output_dim
        self.hidden_layers = hidden_layers
        self.hidden_nodes = hidden_nodes
        self.activation_fn = activation_fn
        self.output_fn = output_fn

        # Regularization params
        self.dropout = dropout
<<<<<<< HEAD
        self.regulariser = mix_l1_l2_reg(reg_params)
=======
        self.regulariser = mix_l1_l2_reg(regulariser_params)
>>>>>>> develop

        # Seed
        self.seed = seed

        # Add intermediate layers to the model
<<<<<<< HEAD
        for layer_id_ in range(1, self.hidden_layers + 1):
=======
        for layer_id_ in range(self.hidden_layers):
>>>>>>> develop
            # Add Dense layer to model
            layer_ = Dense(units=self.hidden_nodes,
                           activation=self.activation_fn,
                           kernel_regularizer=self.regulariser,
                           activity_regularizer=self.regulariser)

            self.__setattr__('layer_' + str(layer_id_), layer_)

<<<<<<< HEAD
        # Add Input and Output Layers
        self.input_layer = Dense(units=self.hidden_nodes, activation=self.activation_fn)
        self.output_layer = Dense(units=self.output_dim, activation=self.output_fn)

        # Dropout layer
        self.dropout_layer = Dropout(rate=self.dropout, seed=self.seed)

    def call(self, inputs, training=True, **kwargs):
=======
            # Add corresponding Dropout Layer
            dropout_layer = Dropout(rate=self.dropout, seed=self.seed + layer_id_)
            self.__setattr__('dropout_layer_' + str(layer_id_), dropout_layer)

        # Input and Output layers
        self.input_layer = Dense(units=self.hidden_nodes, activation=self.activation_fn)
        self.output_layer = Dense(units=self.output_dim, activation=self.output_fn)

    def call(self, inputs, training=True):
>>>>>>> develop
        """Forward pass of layer block."""
        x_inter = self.input_layer(inputs)

        # Iterate through hidden layer computation
        for layer_id_ in range(self.hidden_layers):
<<<<<<< HEAD
            # Get layer and run on input
            layer_ = self.__getattribute__('layer_' + str(layer_id_))
            x_inter = self.dropout_layer(layer_(x_inter, training=training), seed=self.seed)

        return self.output_layer(x_inter, training=training, **kwargs)
=======
            # Get layers
            layer_ = self.__getattribute__('layer_' + str(layer_id_))
            dropout_layer_ = self.__getattribute__('dropout_layer_' + str(layer_id_))

            # Make layer computations
            x_inter = dropout_layer_(layer_(x_inter, training=training))

        return self.output_layer(x_inter, training=training)
>>>>>>> develop

    def get_config(self):
        """Update configuration for layer."""

<<<<<<< HEAD
        # Get overarching configuration
        config = super().get_config().copy()

        # Update configuration with parameters above
=======
        # Load existing configuration
        config = super().get_config().copy()

        # Update configuration
>>>>>>> develop
        config.update({f"{self.name}-output_dim": self.output_dim,
                       f"{self.name}-hidden_layers": self.hidden_layers,
                       f"{self.name}-hidden_nodes": self.hidden_nodes,
                       f"{self.name}-activation_fn": self.activation_fn,
                       f"{self.name}-output_fn": self.output_fn,
                       f"{self.name}-dropout": self.dropout,
                       f"{self.name}-seed": self.seed})

        return config


class FeatTimeAttention(Layer):
    """
<<<<<<< HEAD
    Feature Time Attention Layer. Computes approximations when given output RNN cell states.
=======
    Custom Feature Attention Layer. Features are projected to latent dimension and approximate output RNN states.
    Approximations are sum-weighted to obtain a final representation.
>>>>>>> develop

    Params:
    units: int, dimensionality of projection/latent space.
    activation: str/fn, the activation function to use. (default = "relu")
    name: str, the name on which to save the layer. (default = "custom_att_layer")
    """

<<<<<<< HEAD
    def __init__(self, units: int, activation="relu", name: str = "custom_layer"):
        super().__init__(name=name)

        # Layer hyper-params
=======
    def __init__(self, units, activation="linear", name: str = "custom_layer"):

        # Load layer params
        super().__init__(name=name)

        # Initialise key layer attributes
>>>>>>> develop
        self.units = units
        self.activation_name = activation
        self.activation = tf.keras.activations.get(activation)  # get activation from  identifier

<<<<<<< HEAD
        # Layer kernel and bias initialised to None (updated on build method)
        self.kernel = None
        self.bias = None
        self.unnorm_beta = None

    def build(self, input_shape=None):
        """Build method for initialising layers when seeing input."""

        # Initialise dimensions
        N, T, D_f = input_shape

        # Define Kernel and Bias for Feature Projection
        self.kernel = self.add_weight("kernel", shape=[1, 1, D_f, self.units],
                                      initializer="glorot_uniform", trainable=True)
        self.bias = self.add_weight("bias", shape=[1, 1, D_f, self.units],
                                    initializer='uniform', trainable=True)

        # Define Time aggregation weights for averaging over time.
        self.unnorm_beta = self.add_weight(name='unnorm_beta_weights', shape=[1, T, 1],
                                           initializer="uniform", trainable=True)
=======
        # Initialise layer weights to None
        self.kernel = None
        self.bias = None
        self.unnorm_beta_weights = None

    def build(self, input_shape=None):
        """Build method for the layer given input shape."""
        N, T, Df = input_shape

        # kernel, bias for feature -> latent space conversion
        self.kernel = self.add_weight("kernel", shape=[1, 1, Df, self.units],
                                      initializer="glorot_uniform", trainable=True)
        self.bias = self.add_weight("bias", shape=[1, 1, Df, self.units],
                                    initializer='uniform', trainable=True)

        # Time aggregation learn weights
        self.unnorm_beta_weights = self.add_weight(name='time_agg', shape=[1, T, 1],
                                                   initializer="uniform", trainable=True)
>>>>>>> develop

        super().build(input_shape)

    def call(self, inputs, **kwargs):
        """
<<<<<<< HEAD
        Forward pass of the Feature Time layer - requires inputs and estimated latent projections.

        Params:
        inputs: Tuple of tensors
            Input data - Tensor of shape (batch size, T, Df)
            Latent_reps - Tensor of shape (batch_size, T, latent dim)

        Approximations to latent_reps are computed. At a second stage, approximations are sum-weighted according
        to the beta weights.
        """

        # Load tuple
        input_data, latent_reps = inputs

        # Generate latent vector approximation
        o_hat, _ = self.compute_output_state_approximations(input_data, latent_reps)  # shape (bs, T, latent_dim)

        # Weighted sum over time. Weights are not normalised, hence must be converted through softmax.
        time_weights = _norm_abs(self.unnorm_beta)  # shape (1, T, 1)
        z = tf.reduce_sum(tf.math.multiply(o_hat, time_weights), axis=1)  # shape (bs, latent_dim)

        return z

    def compute_output_state_approximations(self, inputs, latent_reps):
        """Given input and targets (latent_reps), compute OLS approximation to targets and weights."""

        # Compute feature projections
        feature_linear_proj = tf.math.multiply(tf.expand_dims(inputs, axis=-1), self.kernel) + self.bias
        feature_projections = self.activation(feature_linear_proj)

        # Estimate OLS coefficients
        alpha_t = _estimate_alpha(feature_projections, targets=latent_reps)

        # Compute OLS estimates given coefficients
=======
        Forward pass of the Custom layer - requires inputs and estimated latent projections.

        Params:
        - inputs: tuple of two arrays:
            - input_data: array-like of input data of shape (bs, T, D_f)
            - latent_reps: array-like of representations of shape (bs, T, units)

        returns:
        - latent_representation (z): array-like of shape (bs, units)
        """

        # Unpack input
        input_data, latent_reps = inputs

        # Compute output state approximations
        o_hat, _ = self.compute_o_hat_and_alpha(input_data, latent_reps)

        # Normalise temporal weights and sum-weight approximations to obtain representation
        beta_scores = _norm_abs(self.unnorm_beta_weights)
        z = tf.reduce_sum(tf.math.multiply(o_hat, beta_scores), axis=1)

        return z

    def compute_o_hat_and_alpha(self, inputs, latent_reps):
        """
        Compute approximation to latent representations, given input feature data.

        Params:
        - inputs: array-like of shape (bs, T, D_f)
        - latent_reps: array-like of shape (bs, T, units)

        returns:
        - output: tuple of arrays:
           - array-like of shape (bs, T, units) of representation approximations
           - array-like of shape (bs, T, D_f) of alpha_weights
        """

        # Compute feature projections
        feature_projections = self.activation(
            tf.math.multiply(tf.expand_dims(inputs, axis=-1), self.kernel) + self.bias)

        # estimate alpha coefficients through OLS
        alpha_t = _estimate_alpha(feature_projections, targets=latent_reps)

        # sum-weight feature projections according to alpha_t to compute latent approximations
>>>>>>> develop
        o_hat = tf.reduce_sum(tf.math.multiply(tf.expand_dims(alpha_t, axis=-1), feature_projections), axis=2)

        return o_hat, alpha_t

<<<<<<< HEAD
    def compute_unnorm_scores(self, inputs, latent_reps, cluster_reps):
        """
        Compute unnormalised alpha, beta, gamma scores given
        input data
        latent_representation
        cluster_representations (if None, ignore gamma).
        """

        # alpha estimated from OLS regression
        o_hat, alpha_t = self.compute_output_state_approximations(inputs, latent_reps)  # shape bs, T, D_f

        # beta estimated from beta_weights
        beta = self.unnorm_beta  # shape (1, T, 1)

        # gamma estimated from approximation to cluster reps
        if cluster_reps is None:
            gamma_t_k = None
        else:
            gamma_t_k = _estimate_gamma(o_hat, cluster_reps)  # shape (bs, K, T)

        return alpha_t, beta, gamma_t_k

    def compute_norm_scores(self, inputs, latent_reps, cluster_reps):
        """Compute normalised scores alpha, beta, gamma."""

        # Load weights
        alpha, beta, gamma = self.compute_unnorm_scores(inputs, latent_reps, cluster_reps)

        # Normalise
        alpha = _norm_abs(alpha, axis=1)
        beta = _norm_abs(beta, axis=1)

        # Check if gamma is None
        if gamma is None:
            gamma = None
        else:
            gamma = _norm_abs(gamma, axis=1)

        return alpha, beta, gamma
=======
    def compute_unnorm_scores(self, inputs, latent_reps, cluster_reps=None):
        """
        Compute unnorm_weights for attention values.

        Params: - inputs: array-like of shape (bs, T, D_f) of input data - latent_reps: array-like of shape (bs, T,
        units) of RNN cell output states. - cluster_reps: array-like of shape (K, units) of cluster representation
        vectors (default = None). If None, gamma computation is skipped.

        Returns: - output: tuple of arrays (alpha, beta, gamma) with corresponding values. If cluster_reps is None,
        gamma computation is skipped.
        """

        # Compute alpha weights
        o_hat, alpha_t = self.compute_o_hat_and_alpha(inputs, latent_reps)

        # Load beta weights
        beta = self.unnorm_beta_weights

        # If cluster_reps not None, compute gamma
        if cluster_reps is None:
            gamma_t_k = None
        else:
            gamma_t_k = _estimate_gamma(o_hat, cluster_reps)

        return alpha_t, beta, gamma_t_k

    def compute_norm_scores(self, inputs, latent_reps, cluster_reps=None):
        """
        Compute normalised attention scores alpha, beta, gamma.

        Params: - inputs: array-like of shape (bs, T, D_f) of input data - latent_reps: array-like of shape (bs, T,
        units) of RNN cell output states. - cluster_reps: array-like of shape (K, units) of cluster representation
        vectors (default = None). If None, gamma computation is skipped.

        Returns: - output: tuple of arrays (alpha, beta, gamma) with corresponding normalised scores. If cluster_reps
        is None, gamma computation is skipped.
        """

        # Load unnormalised scores
        alpha, beta, gamma = self.compute_unnorm_scores(inputs, latent_reps, cluster_reps)

        # Normalise
        alpha_norm = _norm_abs(alpha, axis=1)
        beta_norm = _norm_abs(beta, axis=1)

        if gamma is None:
            gamma_norm = None
        else:
            gamma_norm = _norm_abs(gamma, axis=1)

        return alpha_norm, beta_norm, gamma_norm
>>>>>>> develop

    def get_config(self):
        """Update configuration for layer."""

<<<<<<< HEAD
        # Get overarching configuration
        config = super().get_config().copy()

        # Update configuration with parameters above
        config.update({f"{self.name}-units": self.units,
                       f"{self.name}-activation": self.activation_name
                       })
=======
        # Load existing configuration
        config = super().get_config().copy()

        # Update configuration
        config.update({f"{self.name}-units": self.units,
                       f"{self.name}-activation": self.activation})
>>>>>>> develop

        return config


class LSTMEncoder(Layer):
    """
        Class for a stacked LSTM layer architecture.

        Params:
        - latent_dim : dimensionality of latent space for each sub-sequence. (default = 32)
        - hidden_layers : Number of "hidden"/intermediate LSTM layers.  (default = 1)
<<<<<<< HEAD
        - hidden_nodes : For hidden LSTM layers, the dimensionality of the intermediate state. (default = 32)
=======
        - hidden_nodes : For hidden LSTM layers, the dimensionality of the intermediate state. (default = 20)
>>>>>>> develop
        - state_fn : The activation function to use on cell state/output. (default = 'tanh')
        - recurrent_activation : The activation function to use on forget/input/output gates. (default = 'sigmoid')
        - return_sequences : Indicates if returns sequence of states on the last layer (default = False)
        - dropout : dropout rate to be used on cell state/output computation. (default = 0.6)
        - recurrent_dropout : dropout rate to be used on forget/input/output gates. (default = 0.0)
        - regulariser_params :  tuple of floats indicating l1_l2 regularisation. (default = (0.01, 0.01))
<<<<<<< HEAD
        - name : Name on which to save component. (default = 'encoder')
        - seed : seed value for dropout layer (default = 4347)
    """

    def __init__(self, latent_dim: int = 32, hidden_layers: int = 1, hidden_nodes: int = 20, state_fn: str = "tanh",
                 recurrent_fn: str = "sigmoid", regulariser_param: tuple = (0.01, 0.01), return_sequences: bool = False,
                 dropout: float = 0.6, recurrent_dropout: float = 0.0, name: str = 'LSTM_Encoder', seed: int = 4347):
=======
        - name : Name on which to save component. (default = 'LSTM_Encoder')
    """

    def __init__(self, latent_dim: int = 32, hidden_layers: int = 1, hidden_nodes: int = 20, state_fn="tanh",
                 recurrent_fn="sigmoid", regulariser_params: tuple = (0.01, 0.01), return_sequences: bool = False,
                 dropout: float = 0.6, recurrent_dropout: float = 0.0, name: str = 'LSTM_Encoder'):
>>>>>>> develop

        # Block Parameters
        super().__init__(name=name)
        self.latent_dim = latent_dim
        self.hidden_layers = hidden_layers
        self.hidden_nodes = hidden_nodes
        self.state_fn = state_fn
        self.recurrent_fn = recurrent_fn
        self.return_sequences = return_sequences

<<<<<<< HEAD
        # Regularisation parameters
        self.dropout = dropout
        self.recurrent_dropout = recurrent_dropout
        self.regulariser_params = regulariser_param
        self.regulariser = mix_l1_l2_reg(regulariser_param)

        # Seed
        self.seed = seed

        # Add Hidden/Intermediate Layers
        for layer_id_ in range(1, self.hidden_layers + 1):
=======
        # Regularisation Params
        self.dropout = dropout
        self.recurrent_dropout = recurrent_dropout
        self.regulariser_params = regulariser_params
        self.regulariser = mix_l1_l2_reg(regulariser_params)

        # Add Intermediate Layers
        for layer_id_ in range(self.hidden_layers):
>>>>>>> develop
            self.__setattr__('layer_' + str(layer_id_),
                             LSTM(units=self.hidden_nodes, return_sequences=True, activation=self.state_fn,
                                  recurrent_activation=self.recurrent_fn, dropout=self.dropout,
                                  recurrent_dropout=self.recurrent_dropout,
                                  kernel_regularizer=self.regulariser, recurrent_regularizer=self.regulariser,
                                  bias_regularizer=self.regulariser, return_state=False))

<<<<<<< HEAD
        # Add Input and Output Layers
=======
        # Input and Output Layers
>>>>>>> develop
        self.input_layer = LSTM(units=self.hidden_nodes, activation=self.state_fn,
                                recurrent_activation=self.recurrent_fn, return_sequences=True,
                                dropout=self.dropout, recurrent_dropout=self.recurrent_dropout,
                                kernel_regularizer=self.regulariser, recurrent_regularizer=self.regulariser,
                                bias_regularizer=self.regulariser, return_state=False)

        self.output_layer = LSTM(units=self.latent_dim, activation=self.state_fn,
                                 recurrent_activation=self.recurrent_fn, return_sequences=self.return_sequences,
                                 dropout=self.dropout, recurrent_dropout=self.recurrent_dropout,
                                 kernel_regularizer=self.regulariser, recurrent_regularizer=self.regulariser,
                                 bias_regularizer=self.regulariser, return_state=False)

    def call(self, inputs, mask=None, training=True):
        """Forward pass of layer block."""
        x_inter = self.input_layer(inputs)

        # Iterate through hidden layer computation
<<<<<<< HEAD
        for layer_id_ in range(1, self.hidden_layers + 1):
=======
        for layer_id_ in range(self.hidden_layers):
>>>>>>> develop
            layer_ = self.__getattribute__('layer_' + str(layer_id_))
            x_inter = layer_(x_inter, training=training)

        return self.output_layer(x_inter, training=training)

    def get_config(self):
        """Update configuration for layer."""

<<<<<<< HEAD
        # Get overarching configuration
        config = super().get_config().copy()

        # Update configuration with parameters above
=======
        # Load existing configuration
        config = super().get_config().copy()

        # Update configuration
>>>>>>> develop
        config.update({f"{self.name}-latent_dim": self.latent_dim,
                       f"{self.name}-hidden_layers": self.hidden_layers,
                       f"{self.name}-hidden_nodes": self.hidden_nodes,
                       f"{self.name}-state_fn": self.state_fn,
                       f"{self.name}-recurrent_fn": self.recurrent_fn,
                       f"{self.name}-return_sequences": self.return_sequences,
                       f"{self.name}-dropout": self.dropout,
                       f"{self.name}-recurrent_dropout": self.recurrent_dropout,
<<<<<<< HEAD
                       f"{self.name}-regulariser_params": self.regulariser_params,
                       f"{self.name}-seed": self.seed})
=======
                       f"{self.name}-regulariser_params": self.regulariser_params})
>>>>>>> develop

        return config


class AttentionRNNEncoder(LSTMEncoder):
    """
<<<<<<< HEAD
        Class for an Attention RNN Encoder architecture.

        Params:
    units: int, dimensionality of projection/latent space.
    activation: str/fn, the activation function to use. (default = "relu")
    name: str, name on which to save model.
    """

    def __init__(self, units: int, activation: str = "linear", name: str = "AttentionRNNEncoder", **kwargs):
        super().__init__(latent_dim=units, return_sequences=True, name=name, **kwargs)
        self.feat_time_att_layer = FeatTimeAttention(units=units, activation=activation)

    def call(self, inputs, mask=None, training=True):
        """Forward pass of layer block."""

        # Compute latent representations through RNN Encoder
        latent_reps = super().call(inputs, mask=mask, training=training)

        # Attention layer inputs
        attention_inputs = inputs, latent_reps
        z = self.feat_time_att_layer(attention_inputs)
=======
        Class for an Attention RNN Encoder architecture. Class builds on LSTM Encoder class.
    """

    def __init__(self, units, activation="linear", **kwargs):
        super().__init__(latent_dim=units, return_sequences=True, **kwargs)
        self.feat_time_attention_layer = FeatTimeAttention(units=units, activation=activation)

    def call(self, inputs, mask=None, training: bool = True):
        """
        Forward pass of layer block.

        Params:
        - inputs: array-like of shape (bs, T, D_f)
        - mask: array-like of shape (bs, T) (default = None)
        - training: bool indicating whether to make computation in training mode or not. (default = True)

        Returns:
        - z: array-like of shape (bs, units)
        """

        # Compute LSTM output states
        latent_reps = super().call(inputs, mask=mask, training=training)

        # Compute representation through feature time attention layer
        attention_inputs = (inputs, latent_reps)
        z = self.feature_time_att_layer(attention_inputs)
>>>>>>> develop

        return z

    def compute_unnorm_scores(self, inputs, cluster_reps=None):
<<<<<<< HEAD
        """
        Compute alpha, beta, gamma un-normalised scores for cluster estimation.

        Params:
        - inputs: input data
        - cluster_reps: set of cluster representation vectors (default = None)
        """
        latent_reps = super().call(inputs, training=False)

        return self.feat_time_att_layer.compute_unnorm_scores(inputs, latent_reps, cluster_reps)

    def compute_norm_scores(self, inputs, cluster_reps=None):
        """
        Compute alpha, beta, gamma normalised scores for cluster estimation.

        Params:
        - inputs: input data
        - cluster reps: set of cluster representation vectors (default = None).
        """

        # Compute latent representations and estimate alpha
        latent_reps = super().call(inputs, training=False)
        alpha, beta, gamma = self.feat_time_att_layer.compute_norm_scores(inputs, latent_reps, cluster_reps)

        return alpha, beta, gamma
=======
        """Compute unnormalised scores alpha, beta, gamma given input data and cluster representation vectors.

        Params:
        - inputs: array-like of shape (bs, T, D_f)
        - cluster_reps: array-like of shape (K, units) of cluster representation vectors. (default = None)

        If cluster_reps is None, compute only alpha and beta weights.

        Returns:
        - Tuple of arrays, containing alpha, beta, gamma unnormalised attention weights.
        """
        latent_reps = super().call(inputs, training=False)

        return self.feature_time_att_layer.compute_unnorm_scores(inputs, latent_reps, cluster_reps)

    def compute_norm_scores(self, inputs, cluster_reps=None):
        """Compute normalised scores alpha, beta, gamma given input data and cluster representation vectors.

        Params:
        - inputs: array-like of shape (bs, T, D_f)
        - cluster_reps: array-like of shape (K, units) of cluster representation vectors. (default = None)

        If cluster_reps is None, compute only alpha and beta weights.

        Returns:
        - Tuple of arrays, containing alpha, beta, gamma normalised attention weights.
        """
        latent_reps = super().call(inputs, training=False)

        return self.feature_time_att_layer.compute_norm_scores(inputs, latent_reps, cluster_reps)
>>>>>>> develop

    def get_config(self):
        """Update configuration for layer."""
        return super().get_config()


In [ ]:
#!/usr/bin/env python3


import tensorflow as tf
from sklearn.cluster import KMeans
from tensorflow.keras import optimizers
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Auxiliary



def _class_weighting(y_true):
    """
    Function to compute inverse class proportion weighting given array of true class assignments.

    Params:
    - y_true: array-like of shape (N, num_outcs) with corresponding one-hot encoding assignment of true class

    Returns:
    - weights: array-like of shape (num_outcs) with weights inversely proportional to number of true class examples.
    """

    class_numbers = tf.reduce_sum(y_true, axis=0)

    # Check no class is missing
    if not tf.reduce_all(class_numbers > 0):
        class_numbers += 1

    # Compute reciprocal
    inv_class_numbers = 1 / tf.reduce_sum(y_true, axis=0)

    return inv_class_numbers / tf.reduce_sum(inv_class_numbers)


class CAMELOT(tf.keras.Model):
    """
    Model Class for CAMELOT architecture.

    Params:

        (General)
    - num_clusters: number of clusters. (default = 10)
    - latent_dim: dimensionality of latent space. (default = 32)
    - output_dim: dimensionality of output space. (default = 4)
    - seed: Seed to run analysis on. (default = 4347)
    - name: Name to give the model. (default = "Camelot")

        (Loss functions)
    - alpha: weighting in cluster entropy. (default = 0.01)
    - beta: weighting in clustering representation separation. (default = 0.01)

        (Regularisation Params)
    - regulariser_params: tuple of l1_l2 float weights. (default = (0.01, 0.01))
    - dropout: float corresponding to dropout value. (default = 0.6)

        (Encoder Params)
    - encoder_params: Dictionary indicating parameters for Encoder architecture, as follows:
            - activation: activation function of custom feature projection component (default = "linear")
            - hidden_layers: Number of "hidden"/intermediate LSTM layers.  (default = 1)
            - hidden_nodes: Dimensionality of the intermediate state computation. (default = 20)
            - state_fn: The activation function to use on cell state/output. (default = 'tanh')
            - recurrent_activation: The activation function to use on F/I/O gates. (default = 'sigmoid')
            - recurrent_dropout: dropout rate to be used on forget/input/output gates. (default = 0.0)
    Default value is {}, which resets to default parameters.

        (Identifier Params)
    - identifier_params: Dictionary indicating parameters for Identifier block, as follows:
        - hidden_layers: int, Number of "hidden" feedforward layers. (default = 2)
        - hidden_nodes: int, For hidden feedforward layers, the dimensionality of the output space. (default = 30)
        - activation_fn: str/fn, The activation function to use. (default = 'sigmoid')
    Default value is {"name": "Identifier"}, which resets to default parameters.

        (Predictor Params)
    - predictor_params: Dictionary indicating parameters for Predictor block, as follows:
        - hidden_layers: int, Number of "hidden" feedforward layers. (default = 2)
        - hidden_nodes: int, For hidden feedforward layers, the dimensionality of the output space. (default = 30)
        - activation_fn: str/fn, The activation function to use. (default = 'sigmoid')
    Default value is {"name": "Predictor"}, which resets to default parameters.

        (Cluster Representation Params)
    - cluster_rep_lr: Learning rate for update of cluster_representations. (default = 0.01)

        (Others)
    - optimizer_init: optimizer to use for initialisation training. (default = "adam")
    - weighted_loss: whether to use weights on predictive clustering loss (default = "True")
    """

    def __init__(self, num_clusters=10, latent_dim=32, seed=4347, output_dim=4, name="Camelot",
                 alpha=0.01, beta=0.01, regulariser_params=(0.01, 0.01), dropout=0.6,
                 encoder_params=None, identifier_params=None, predictor_params=None, cluster_rep_lr=0.001,
                 optimizer_init="adam", weighted_loss=True, **kwargs):

        super().__init__(name=name)

        # General params
        self.K = num_clusters
        self.latent_dim = latent_dim
        self.output_dim = output_dim
        self.seed = seed

        # Loss function params
        self.alpha = alpha
        self.beta = beta

        # Common to all Networks
        self.regulariser = regulariser_params
        self.dropout = dropout

        # Build Networks
        self.encoder_params = encoder_params if encoder_params is not None else {}
        self.identifier_params = identifier_params if identifier_params is not None else {}
        self.predictor_params = predictor_params if predictor_params is not None else {}

        self.Encoder = AttentionRNNEncoder(units=self.latent_dim, dropout=self.dropout,
                                           regulariser_params=self.regulariser, name="Encoder",
                                           **self.encoder_params)
        self.Identifier = MLP(output_dim=self.K, dropout=self.dropout, output_fn="softmax",
                              regulariser_params=self.regulariser, seed=self.seed, name="Identifier",
                              **self.identifier_params)
        self.Predictor = MLP(output_dim=self.output_dim, dropout=self.dropout, output_fn="softmax",
                             regulariser_params=self.regulariser, seed=self.seed, name="Predictor",
                             **self.predictor_params)

        # Cluster Representation params
        self.cluster_rep_set = tf.Variable(initial_value=tf.zeros(shape=[self.K, self.latent_dim], dtype='float32'),
                                           trainable=True, name='cluster_rep')
        self.cluster_rep_lr = cluster_rep_lr
        self.cluster_opt = optimizers.Adam(learning_rate=self.cluster_rep_lr)

        # Initialisation loss trackers
        self._enc_pred_loss_tracker = None
        self._iden_loss_tracker = None

        # Others
        self._optimizer_init = tf.keras.optimizers.get(optimizer_init)
        self.weighted_loss = weighted_loss
        self.loss_weights = None

    # Build and Call Methods
    def build(self, input_shape):
        """Build method to serialise layers."""
        self.Encoder.build(input_shape)
        self.Encoder.feat_time_attention_layer.build(input_shape)

        super().build(input_shape)

    def call(self, inputs, **kwargs):
        """
        Call method for model.

        Params:
        - inputs: array-like of shape (bs, T, D_f)

        Returns: tuple of arrays:
            - y_pred: array-like of shape (bs, outcome_dim) with probability assignments.
            - pi: array-like of shape (bs, K) of cluster probability assignments.
        """
        y_pred, pi = self.forward_pass(inputs)

        return y_pred

    # TRAINING RELATED METHODS
    def forward_pass(self, inputs):
        """
        Single forward pass given input data. Inputs are encoded. Encoded vectors pass through Identifier and assigned to clusters. Cluster representations are used to predict outcome.

        Params:
        - inputs: array-like of shape (bs, T, D_f) of input data.https://en.wikipedia.org/wiki/Schauder_basis

        Returns: tuple of arrays:
            - y_pred: array-like of shape (bs, outcome_dim) with probability assignments.
            - pi: array-like of shape (bs, K) of cluster probability assignments.
        """

        z = self.Encoder(inputs)
        pi = self.Identifier(z)

        # Sample from cluster assignments and assign corresponding cluster representations
        cluster_samp = self._sample_from_probs(pi)
        sample_representations = self._select_representations_from_sample(cluster_samp)

        # Make predictions
        y_pred = self.Predictor(sample_representations)

        return y_pred, pi

    def _sample_from_probs(self, clus_probs):
        """
        Method to sample cluster given cluster assignment probabilities and categorical sampling.

        Params:
        - clus_probs: array-like of shape (bs, K) of cluster assignment probabilities.

        Returns:
            - output: array-like of shape (bs,) with the corresponding sampled cluster.
        """

        # Convert to logits
        logits = tf.math.log(tf.reshape(clus_probs, shape=[-1, self.K]))

        # Categorical sampling
        sample = tf.random.categorical(logits, num_samples=1, seed=self.seed)

        return tf.squeeze(sample)

    def _select_representations_from_sample(self, clus_samp):
        """
        Method to select cluster representation vector given cluster assignment.

        Params:
        - cluster_samples: array-like of shape (bs, ) with the corresponding cluster.

        Returns:
            - output: array-like of shape (bs, latent_dim) with the corresponding cluster representation vector.
        """

        # Convert to one-hot encoding
        idx_mask = tf.one_hot(clus_samp, depth=self.K)

        # Obtain representation
        clus_rep = tf.linalg.matmul(idx_mask, self.cluster_rep_set)

        return clus_rep

    def train_step(self, inputs):
        """
        Method to train model.

        Params:
            - inputs: tuple of input (X, y) data:
                - X: array-like of shape (bs, T, D_f) of input time-series feature data.
                - y: array-like of shape (bs, num_outcs) of input outcome class data.

        Method updates model weights according to loss backpropagation.
        """

        # Unpack inputs
        x, y = inputs

        # Define variables for each network
        pred_vars = [var for var in self.trainable_variables if 'Predictor' in var.name]
        enc_id_vars = [var for var in self.trainable_variables if 'Encoder' in var.name or 'Identifier' in var.name]
        rep_vars = self.cluster_rep_set

        # Initialise GradientTape to compute gradients
        with tf.GradientTape(watch_accessed_variables=True, persistent=True) as tape:
            y_pred, pi = self.forward_pass(x)

            if self.weighted_loss is True:
                self.loss_weights = _class_weighting(y)

            # compute losses
            l_pred = model_utils.l_pred(y, y_pred, weights=self.loss_weights)
            l_enc_id = model_utils.l_pred(y, y_pred, weights=self.loss_weights) + self.alpha * model_utils.l_dist(pi)
            l_clus = model_utils.l_pred(y, y_pred, weights=self.loss_weights) + self.beta * model_utils.l_clus(rep_vars)

        # Compute gradients
        pred_grad = tape.gradient(target=l_pred, sources=pred_vars)
        enc_id_grad = tape.gradient(target=l_enc_id, sources=enc_id_vars)
        clus_grad = tape.gradient(target=l_clus, sources=rep_vars)

        # Apply gradients
        self.optimizer.apply_gradients(zip(pred_grad, pred_vars))
        self.optimizer.apply_gradients(zip(enc_id_grad, enc_id_vars))
        self.cluster_opt.apply_gradients(zip([clus_grad], [rep_vars]))

        return l_pred, l_enc_id, l_clus
        # return {'L1': L1.result(), 'L2': L2.result(), 'L3': L3.result()}

    def test_step(self, inputs):
        """
        Method to compute test step model.

        Params:
            - inputs: tuple of input (X, y) data:
                - X: array-like of shape (bs, T, D_f) of input time-series feature data.
                - y: array-like of shape (bs, num_outcs) of input outcome class data.
        """
        x, y = inputs
        y_pred, pi = self.forward_pass(x)

        if self.weighted_loss is True:
            self.loss_weights = _class_weighting(y)

        # compute losses
        l_pred = model_utils.l_pred(y, y_pred, weights=self.loss_weights)
        l_enc_id = model_utils.l_pred(y, y_pred, weights=self.loss_weights) + self.alpha * model_utils.l_dist(pi)
        l_clus = model_utils.l_pred(y, y_pred, weights=self.loss_weights) + self.beta * model_utils.l_clus(
            self.cluster_rep_set)

        return l_pred, l_enc_id, l_clus
        # return {'L1': val_L1.result(), 'L2': val_L2.result(), 'L3': val_L3.result()}

    # Initialisation procedure for the model
    def initialise_model(self, data: tuple, val_data: tuple, epochs: int = 100, learning_rate: float = 0.001,
                         batch_size: int = 64, **kwargs):
        """
        Initialisation Method for Model.

        Params:
            - data: tuple (X, y) of data to train the model.
                - X: array-like of shape (N, T, D_f)
                - y: array-like of shape (N, num_outcs)
            - val_data: tuple (X, y) of validation data to see loss performance.
                - X: array-like of shape (N', T, D_f)
                - y: array-like of shape (N', num_outcs)
            - epochs: int, number of epochs for training. (default = 100)
            - learning_rate: float, starting learning rate for initialisation training
            - batch_size: int, size of individual batches.
            - kwargs: dictionary arguments for KMeans initialisation.

        Updates model according to initialisation procedure. Initialisation consists of 3 steps:
        - Outcome prediction by applying Predictor network directly on input data representation..
        - Cluster representation initialisation through K-Means on input data representation.
        - Identifier initialisation by minimising loss on clusters as predicted by KMeans.
        """

        # Unpack data
        x, y = data
        val_x, val_y = val_data

        # Compute loss weights if necessary
        if self.weighted_loss is True:
            self.loss_weights = _class_weighting(y)

        # Initialise init learning rate
        self._optimizer_init.learning_rate = learning_rate

        # Go through initialisation steps
        self._initialise_enc_pred(data=data, val_data=val_data, epochs=epochs, batch_size=batch_size)
        clus_train_y, clus_val_y = self._initialise_clus(x, val_x, **kwargs)

        # Initialise Identifier
        iden_train_data = (x, clus_train_y)
        iden_val_data = (val_x, clus_val_y)
        self._initialise_iden(data=iden_train_data, val_data=iden_val_data,
                              epochs=epochs, batch_size=batch_size)

    def _initialise_enc_pred(self, data, val_data, epochs=100, batch_size=64):
        """
          Initialisation Method for Encoder and Predictor blocks.

          Params:
              - data: tuple (X, y) of data to train the model.
                  - X: array-like of shape (N, T, D_f)
                  - y: array-like of shape (N, num_outcs)
              - val_data: tuple (X, y) of validation data to see loss performance.
                  - X: array-like of shape (N', T, D_f)
                  - y: array-like of shape (N', num_outcs)
              - epochs: int, number of epochs for training. (default = 100)
              - batch_size: int, size of individual batches.

        Input data passes through Encoder network to obtain data representations. Predictor then outputs a predicted class. This is matched against the true class.
        """

        # Unpack inputs
        x, y = data
        x_val, y_val = val_data

        # Convert to tf.Dataset for iteration
        inputs = tf.data.Dataset.from_tensor_slices((x, y)).shuffle(buffer_size=5000).batch(batch_size)

        # Initialise loss tracker
        self._enc_pred_loss_tracker = pd.DataFrame(data=np.nan, index=range(epochs), columns=["train_loss", "val_loss"])
        enc_pred_vars = [var for var in self.trainable_variables if "Encoder" in var.name or "Predictor" in var.name]

        # Iterate through epochs and batches
        print("-" * 20, "\n", "Initialising encoder-predictor training.")
        for epoch in range(epochs):

            epoch_loss = 0
            for step_, (x_batch, y_batch) in enumerate(inputs):
                # One Training Step
                with tf.GradientTape(watch_accessed_variables=False) as tape:
                    tape.watch(enc_pred_vars)

                    # Prediction and loss
                    y_pred = self.Predictor(self.Encoder(x_batch))
                    loss_batch = model_utils.l_pred(y_batch, y_pred, weights=self.loss_weights)

                # Update gradients
                enc_pred_grad = tape.gradient(loss_batch, enc_pred_vars)
                self._optimizer_init.apply_gradients(zip(enc_pred_grad, enc_pred_vars))

                # Update loss
                epoch_loss += loss_batch

                # Print current batch loss - clears line and re-writes
                print("Batch Loss %.4f" % loss_batch, end="\r", flush=True)

            # Compute validation loss on validation data
            y_val_pred = self.Predictor(self.Encoder(x_val))
            loss_val = model_utils.l_pred(y_val, y_val_pred, weights=self.loss_weights)

            # Print result and update tracker
            print("End of epoch %d - \n Training loss: %.4f  Validation loss %.4f" % (
            epoch, epoch_loss / step_, loss_val))
            self._enc_pred_loss_tracker.loc[epoch, :] = [epoch_loss / step_, loss_val]

            # Check if has improved or not
            if self._enc_pred_loss_tracker.iloc[-50:, -1].le(loss_val + 0.001).any():
                break

    def _initialise_clus(self, x, val_x, **kwargs):
        """
          Initialisation Method for cluster representation

          Params:
              - x: array-like of shape (N, T, D_f)
              - val_x: array-like of shape (N', T, D_f)
              - kwargs: other arguments relevant to KMeans method.

        Cluster representations are initialised through KMeans on the set of data representations.

        Returns:
            - tuple:
                - clus_train_y: array-like of shape (N, num_clus) of cluster one-hot assignments.
                - clus_val_y: array-like of shape (N', num_clus) of cluster one-hot assignments.
        """

        # Compute Latent Projections
        print("-" * 20, "\n", "Initialising cluster representations.")
        z = self.Encoder(x).numpy()

        # Fit KMeans
        km = KMeans(n_clusters=self.K, init="k-means++", random_state=self.seed, **kwargs)
        km.fit(z)
        print("KMeans fit has completed.")

        # Centers are figureholders for representations and
        centers = km.cluster_centers_
        cluster_pred = km.predict(z)

        # Compute Initialised Estimates
        print("\nInitialised Phenotypes: ", self.Predictor(centers))
        print("\nEstimated Cluster distribution: ", np.unique(cluster_pred, return_counts=True))

        # Initialise embeddings and convert to one-hot encoding for Identifier
        self.cluster_rep_set.assign(tf.convert_to_tensor(centers, name='cluster_rep', dtype='float32'))
        clus_train_y = np.eye(self.K)[cluster_pred]

        # Make predictions on validation data
        z_val = self.Encoder(val_x).numpy()
        clus_val_y = np.eye(self.K)[km.predict(z_val)]

        return clus_train_y.astype(np.float32), clus_val_y.astype(np.float32)

    def _initialise_iden(self, data, val_data, epochs=100, batch_size=64):
        """
          Initialisation Method for Identifier Network

          Params:
              - data: tuple (X, clus_train_y) of data to train the model.
                  - X_train: array-like of shape (N, T, D_f)
                  - clus_train_y: array-like of shape (N, K)
              - val_data: tuple (X, clus_val_y) of validation data to see loss performance.
                  - X_train: array-like of shape (N', T, D_f)
                  - clus_val_y: array-like of shape (N', K)
              - epochs: int, number of epochs for training. (default = 100)
              - batch_size: int, size of individual batches.

        Input data passes through Encoder network to obtain data representations. Predictor then outputs a predicted class. This is matched against the true class.
        """
        # Input in the right format
        X_train, clus_train_y = data
        X_val, clus_val_y = val_data

        # Convert to tf.Dataset for iteration
        inputs = tf.data.Dataset.from_tensor_slices((X_train, clus_train_y)).shuffle(buffer_size=5000).batch(batch_size)

        # Initialise loss tracker
        self._iden_loss_tracker = pd.DataFrame(data=np.nan, index=range(epochs), columns=["train_loss", "val_loss"])
        iden_vars = [var for var in self.trainable_variables if "Identifier" in var.name]

        # Forward Identifier pass and train
        print("-" * 20, "\nInitialising Identifier training.")

        for epoch in range(epochs):  # Iterate through epochs
            epoch_loss = 0
            for step_, (x_batch, clus_batch) in enumerate(inputs):  # Iterate through batch

                # One Training Step
                with tf.GradientTape(watch_accessed_variables=False) as tape:
                    tape.watch(iden_vars)

                    # Prediction and loss
                    clus_pred = self.Identifier(self.Encoder(x_batch))
                    loss_batch = model_utils.l_pred(clus_batch, clus_pred)

                # Update gradients
                iden_grad = tape.gradient(loss_batch, iden_vars)
                self._optimizer_init.apply_gradients(zip(iden_grad, iden_vars))

                # Update loss
                epoch_loss += loss_batch

                # Print current batch loss - clears line and re-writes
                print("Batch Loss %.4f" % loss_batch, end="\r", flush=True)

            # Compute validation loss on validation data
            clus_val_pred = self.Identifier(self.Encoder(X_val))
            loss_val = model_utils.l_pred(clus_val_y, clus_val_pred)

            # Print result and update tracker
            print("End of epoch %d - \n Training loss: %.4f  Validation loss %.4f" % (
            epoch, epoch_loss / step_, loss_val))
            self._iden_loss_tracker.loc[epoch, :] = [epoch_loss / step_, loss_val]

            # Check if has improved or not - look at last 50 epoch validation loss and check if 
            if self._iden_loss_tracker.iloc[-50:, -1].le(loss_val + 0.001).any():
                break

    # USEFUL METHODS

    def compute_unnorm_attention_weights(self, inputs):
        """
        Computes unnormalised attention weights alpha, beta, gamma.

        Params:
        - inputs: array-like of shape (N, T, D_f).

        Return: tuple of unnormalised attention weights.
            - alpha: array-like of shape (N, T, D_f)
            - beta: array-like of shape (1, T, 1)
            - gamma: array-like of shape (N, K, D_f)
        """
        scores = self.Encoder.compute_unnorm_scores(inputs, cluster_reps=self.cluster_rep_set)

        return scores

    def compute_norm_attention_weights(self, inputs):
        """
        Computes normalised attention weights alpha, beta, gamma.

        Params:
        - inputs: array-like of shape (N, T, D_f).

        Return: tuple of normalised attention weights.
            - alpha: array-like of shape (N, T, D_f)
            - beta: array-like of shape (1, T, 1)
            - gamma: array-like of shape (N, K, D_f)
        """
        scores = self.Encoder.compute_norm_scores(inputs, cluster_reps=self.cluster_rep_set)

        return scores

    def compute_cluster_phenotypes(self):
        """
        Compute Cluster Phenotypes given cluster representations.
        """
        phens = self.Predictor(self.cluster_rep_set).numpy()

        return phens

    def clus_assign(self, X):
        """
        Compute cluster assignments given input data X.

        Params:
        - X: array-like of shape (bs, T, D_f)

        Returns:
        - clus_pred: array-like of shape (bs, ) with corresponding cluster assignment.
            """
        pi = self.Identifier(self.Encoder(X)).numpy()
        clus_pred = np.argmax(pi, axis=1)

        return clus_pred

    def get_cluster_reps(self):
        return self.cluster_rep_set.numpy()

    def compute_pis(self, X):
        """Obtain cluster assignment probabilities."""
        pis = self.Identifier(self.Encoder(X))

        return pis.numpy()

    def get_config(self):
        """Update configuration for layer."""
        config = super().get_config().copy()
        config.update({"latent_dim": self.latent_dim, "hidden_layers": self.hidden_layers,
                       "hidde_nodes": self.hidden_nodes, "state_fn": self.state_fn,
                       "recurrent_fn": self.recurrent_fn, "return_sequences": self.return_sequences,
                       "dropout": self.dropout, "recurrent_dropout": self.recurrent_dropout,
                       "regulariser_params": self.regulariser_params})
        """Update configuration for layer."""

        # Load existing configuration
        config = super().get_config().copy()

        # Update configuration
        config.update({f"{self.name}-K": self.K,
                       f"{self.name}-latent_dim": self.latent_dim,
                       f"{self.name}-output_dim": self.output_dim,
                       f"{self.name}-seed": self.seed,
                       f"{self.name}-alpha": self.alpha,
                       f"{self.name}-beta": self.beta,
                       f"{self.name}-regulariser": self.regulariser,
                       f"{self.name}-dropout": self.dropout,
                       f"{self.name}-weighted_loss": self.weighted_loss})

        return config


class Model:
    """
    Class for fitting and evaluating Camelot architecture model.
    """

    def __init__(self, data_info, model_config):
        "Initialise model configuration parameters."
        self.data_info = data_info
        self.model_config = model_config
        self.model = None

    def fit(self, train_params):
        """
        Fit method for training CAMELOT model.

        Params:
        - train_params: dictionary containing trianing parameter information:
            - "lr": learning rate for training
            - "epochs_init": number of epochs to train initialisation
            - "epochs": number of epochs for main training
            - "bs": batch size
            - "cbck_str": callback_string indicating which callbacks to print during training
        """

        # Unpack relevant data information
        X_train, X_val, X_test = self.data_info["X"]
        y_train, y_val, y_test = self.data_info["y"]
        output_dim = self.data_info["output_dim"]

        # Unpack training parameters
        lr = train_params["lr"]
        epochs = train_params["epochs"]
        bs = train_params["bs"]
        callback_str = train_params["cbck_str"]

        # TODO- ADD GPU USAGE

        # Initialise model
        self.model = CAMELOT(output_dim=output_dim, **self.model_config)
        self.model.build(X_train.shape)

        # Load optimizer
        optimizer = optimizers.Adam(learning_rate=lr)
        self.model.compile(optimizer=optimizer, run_eagerly=True)

        # Train model on initialisation procedure
        epochs_init = 100
        print("-" * 20, "\n", "Initialising Model", sep="\n")
        self.model.initialise_model(data=(X_train, y_train), val_data=(X_val, y_val), epochs=epochs, learning_rate=lr,
                                    batch_size=bs)

        # Main Training phase
        print("-" * 20, "\n", "STARTING MAIN TRAINING PHASE")
        callbacks, run_num = model_utils.get_callbacks(track_loss="L1", other_cbcks=callback_str, early_stop=True,
                                                       lr_scheduler=True,
                                                       tensorboard=True)

        self.model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=bs, epochs=epochs, verbose=1,
                       callbacks=callbacks)

    def evaluate(self, data_test):
        """
        Evaluation method for computing output results.

        Params:
        - data_test: tuple of array-like test data:
            - X_test: of shape (?, T, D_f)
            - y_test: of shape (?, num_outcs)

        Returns:

        """

        # Unpack test data
        X_test, y_test = data_test

        # Source outcome names and patient id info
        id_info = self.data_info["ids"][-1]
        pat_ids = id_info[:, 0, 0]
        outc_dims = self.data_info["outc_dims"]

        # Other useful defs
        K = self.model.K
        cluster_names = list(range(1, K + 1))

        # Firstly, compute predicted y estimates
        y_pred = pd.DataFrame(self.model.predict(X_test).numpy(), index=pat_ids, columns=outc_dims)
        outc_pred = pd.Series(np.argmax(y_pred, axis=-1), index=pat_ids)
        y_true = pd.Series(y_test, index=pat_ids)


        # Secondly, compute predicted cluster assignments
        pis_pred = pd.DataFrame(self.model.compute_pis(X_test).numpy(), index=pat_ids, columns=cluster_names)
        clus_pred = pd.Series(self.model.clus_assign(X_test).numpy(), index=pat_ids)

        # Thirdly, compute cluster phenotype information
        clus_phenotypes = pd.DataFrame(self.model.compute_cluster_phenotypes(), index=cluster_names, columns=outc_dims)
        cluster_rep_set = self.model.get_cluster_reps()

        # Fourth, save model init losses
        init_loss_1 = self.model._enc_pred_loss_tracker
        init_loss_2 = self.model._iden_loss_tracker
        enc_pred_tracker.index.name, iden_tracker.index.name = "epoch", "epoch"


        # Save data
        y_pred.to_csv(save_fd + "y_pred.csv", index=True, header=True)
        outc_pred.to_csv(save_fd + "outc_pred.csv", index=True, header=True)
        y_true.to_csv(save_fd + "y_true.csv", index=True, header=True)
        pis_pred.to_csv(save_fd + "pis_pred.csv", index=True, header=True)
        clus_pred.to_csv(save_fd + "clus_pred.csv", index=True, header=True)
        clus_phenotypes.to_csv(save_fd + "clus_phenotypes.csv", index=True, header=True)
        np.save(save_fd + "cluster_representations.npy", cluster_rep_set, allow_pickle=True)

        # save losses and model params
        init_loss_1.to_csv(track_fd + "enc_pred_init_loss.csv", index=True, header=True)
        init_loss_2.to_csv(track_fd + "iden_init_loss.csv", index=True, header=True)


        # save parasm
        # Save Model Configuration on both results and experiments
        # with open(save_fd + "config", "w+") as f:
        #     json.dump(vars(params), f)
        #     f.close()

        # with open(track_fd + "config", "w+") as f:
        #     json.dump(vars(params), f)
        #     f.close()

        # Evaluate results
        # auc, f1, rec = os.system("""python run_model.py --K {} --latent_dim {} --seed {}""".format(
        #     K, latent_dim, seed))

        # ------------------------------------------------------------------------------
        """Finally, print some basic statistics for analysing this run"""
        y_true = y_test
        y_pred = y_pred.values
        # auc, f1, rec, pur = utils.super_scores(y_true, y_pred)
        # sil, sil_avg, dbi, dbi_avg, vri, vri_avg = utils.unsuper_scores(X_test, clusters_pred)

        # print("Supervised Performance:", f"AUC: {auc:.2f}", f"f1: {f1:.2f}", f"rec: {rec:.2f}", f"pur: {pur:.2f}", sep = "\n")
        # print("Unsupervised Scores:", f"SIL: {sil:.2f}, {sil_avg:.2f}", f"DBI: {dbi:.2f}, {dbi_avg:.2f}",
        #       f"vri: {vri:.2f} {vri_avg:.2f}", sep = "\n")

        cluster_dist = pd.Series(data=0, index=cluster_names)
        for clus in cluster_names:
            cluster_dist.loc[clus] = np.sum(clusters_pred == clus)

        print("Cluster Assignment distribution: ", cluster_dist, sep="\n")
        print("Num Clusters with patients: {}".format(np.sum(cluster_dist != 0)))

        outcome_df = pd.read_csv("data/HAVEN/processed/copd_outcomes.csv", index_col=0)
        outcomes = outcome_df.loc[clusters_pred.index, :]

        for clus in cluster_names:
            print("Outcome distribution in Cluster {}".format(clus))

            outcome_distribution = outcomes[clusters_pred == clus].sum(axis=0)

            print(outcome_distribution)

        alpha_sc, beta_sc, gamma_sc = model.compute_attention_rnn_encoder_scores(X_test)
        np.savez(save_fd + "attention-all.npz", alpha=alpha_sc, beta=beta_sc, gamma=gamma_sc)

       


Training models

In [ ]:
training_config={
   "lr_init": 0.002,
   "lr": 0.001,
   "epochs_init_1": 30,
   "epochs_init_2": 30,
   "epochs": 30,
   "bs": 64,
   "cbck_str": "",
   "patience_epochs": 200,
   "gpu": 0
}


In [ ]:
"Data Loading."
data_info = data_loader(**data_config)



# -------------------------- Loading and Training Model -----------------------------

"Load model and fit"
print("\n\n\n\n")
model = model_utils.get_model_from_str(data_info=data_info, model_config=model_config,
                                       training_config=training_config)
# Train model
history = model.train(data_info=data_info, **training_config)

"Compute results on test data"
outputs_dic = model.analyse(data_info)
print(outputs_dic.keys())

# -------------------------------------- Evaluate Scores --------------------------------------

"Evaluate scores on the resulting models. Note X_test is converted back to input dimensions."
scores = evaluate(**outputs_dic, data_info=data_info, avg=None)

Results comparison:

In [ ]:

import matplotlib.pyplot as plt

from typing import Union, List

from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, confusion_matrix
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, average_precision_score
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay
from sklearn.metrics.cluster import contingency_matrix


def get_clus_outc_numbers(y_true: np.ndarray, clus_pred: np.ndarray):
    """
    Compute contingency matrix: entry (i,j) denotes the number of patients with true sample i and predicted clus j.

    Params:
    - y_true: array-like of true outcome one-hot assignments.
    - clus_pred: array-like of cluster label assignments.

    Returns:
        - cont_matrix: numpy ndarray of shape (num_outcs, num_clus) where entry (i,j) denotes the number of patients
        with outcome i and cluster j.
    """

    # Convert to categorical
    labels_true = np.argmax(y_true, axis=1)
    labels_pred = clus_pred

    # Compute contingency matrix
    cont_matrix = contingency_matrix(labels_true, labels_pred)

    return cont_matrix


def _convert_to_one_hot_from_probs(array_pred: Union[np.ndarray, pd.DataFrame]):
    """
    Convert array of predicted class/cluster probability assignments to one-hot encoding of the most common class/clus.

    Params: - array_pred: array-like of shape (N, K), where K is the number of target classes, with probability class
    assignments.

    Returns:
    - Output: array-like of shape (N, K) with one-hot encoded most likely class assignments.
    """

    # Convert to array if necessary
    if isinstance(array_pred, pd.DataFrame):
        array_pred = array_pred.values

    # Compute dimensionality
    if len(array_pred.shape) == 2:
        _, K = array_pred.shape

        # Convert to categorical
        class_pred = np.eye(K)[np.argmax(array_pred, axis=1)]

    else:
        # Array_pred already categorical
        K = array_pred.size

        # Convert to categorical
        class_pred = np.eye(K)[array_pred]

    return class_pred


def purity(y_true: np.ndarray, clus_pred: np.ndarray) -> float:
    """
    Computes Purity Score from predicted and true outcome labels. Purity Score is an external cluster validation tool
    which computes the largest number of individuals from a given class in a given cluster, and consequently averages
    this values over the number of clusters.

    Params:
    - y_true: array-like of shape (N, num_outcs) of true outcome labels in one-hot encoded format.
    - clus_pred: array-like of shape (N, num_clus) of predicted outcome cluster assignments.

    Returns:
    - purity_score: float indicating purity score.
    """

    # Convert clus_pred to categorical cluster assignments
    cm = get_clus_outc_numbers(y_true, clus_pred)  # shape (num_outcs, num_clus)

    # Number of most common class in each cluster
    max_class_numbers = np.amax(cm, axis=0)

    # Compute average
    purity_score = np.sum(max_class_numbers) / np.sum(cm)

    return purity_score


def compute_supervised_scores(y_true: np.ndarray, y_pred: np.ndarray, avg=None, outc_names=None):
    """
    Compute set of supervised classification scores between y_true and y_pred. List of metrics includes:
    a) AUROC, b) Recall, c) F1, d) Precision, e) Adjusted Rand Index and f) Normalised Mutual Information Score.

    Params:
    - y_true: array-like of shape (N, num_outcs) of one-hot encoded true class membership.
    - y_pred: array-like of shape (N, num_outcs) of predicted outcome probability assignments.
    - avg: parameter for a), b), c) and d) computation indicating whether class scores should be averaged, and how.
    (default = None, all scores reported).
    - outc_names: List or None, name of outcome dimensions.

    Returns:
        - Dictionary of performance scores:
            - "ROC-AUC": list of AUROC One vs Rest values.
            - "Recall": List of Recall One vs Rest values.
            - "F1": List of F1 score One vs Rest values.
            - "Precision": List of Precision One vs Rest values.
            - "ARI": Float value indicating Adjusted Rand Index performance.
            - "NMI": Float value indicating Normalised Mutual Information Score performance.
    """
    num = 10000

    if isinstance(y_pred, pd.DataFrame):
        y_pred = y_pred.values

    if isinstance(y_true, pd.DataFrame):
        y_true = y_true.values


    # # Compute AUROC and AUPRC, both custom and standard
    # auroc, auprc = bin_utils.custom_auc_auprc(y_true, y_pred, mode="OvR", num=num).values()
    # auroc_custom, auprc_custom = bin_utils.custom_auc_auprc(y_true, y_pred, mode="custom", num=num).values()

    # # GET ROC AND PRC CURVES
    # roc_prc_curves = {
    #     "OvR": bin_utils.plot_auc_auprc(y_true, y_pred, mode="OvR", outc_names=outc_names, num=num),
    #     "Custom": bin_utils.plot_auc_auprc(y_true, y_pred, mode="custom", outc_names=outc_names, num=num)
    # }
    auroc = roc_auc_score(y_true, y_pred, average=None)
    
    # Convert input arrays to categorical labels
    labels_true, labels_pred = np.argmax(y_true, axis=1), np.argmax(y_pred, axis=1)

    # Compute F1
    f1 = f1_score(labels_true, labels_pred, average=avg)

    # Compute Recall
    rec = recall_score(labels_true, labels_pred, average=avg)

    # Compute Precision
    prec = precision_score(labels_true, labels_pred, average=avg)

    # Compute ARI
    ari = adjusted_rand_score(labels_true, labels_pred)

    # Compute NMI
    nmi = normalized_mutual_info_score(labels_true, labels_pred)

    # Compute Confusion matrix
    cm = confusion_matrix(y_true=labels_true, y_pred=labels_pred, labels=None, sample_weight=None, normalize=None)

    # Return Dictionary
    scores_dic = {
        "ROC-AUC": auroc,
        # "ROC-PRC": auprc,
        # # "ROC-AUC-custom": auroc_custom,
        # # "ROC-PRC-custom": auprc_custom,
        "F1": f1,
        "Recall": rec,
        "Precision": prec,
        "ARI": ari,
        "NMI": nmi
    }

    # return scores_dic, cm, roc_prc_curves
    return scores_dic, cm, None


def compute_from_eas_scores(y_true: np.ndarray, scores: np.ndarray, outc_names: np.ndarray = None, **kwargs) -> dict:
    """
    Compute supervised performance metrics given input array scores.


    Params:
    - y_true: array-like of shape (N, num_outcs).
    - scores: array-like of shape (N, ).
    - outc_names: array-like of shape (num_outcs, ) with particular outcome names.
    - kwargs: any other arguments. They are kept for coherence.

    Returns:
    - dict with scores ROC-AUC, F1, Recall, Precision per class
    """

    # Useful info
    num_outcs = y_true.shape[-1]

    if outc_names is None:
        outc_names = range(num_outcs)

    # Useful info and initialise output
    SCORE_NAMES = {"ROC-AUC": roc_auc_score, "F1": f1_score, "Recall": recall_score, "Precision": precision_score}
    output_dic = {}

    # Convert to useful format
    if isinstance(scores, pd.Series) or isinstance(scores, pd.DataFrame):
        scores = scores.values.reshape(-1)

    # Convert scores to probability thresholds
    scores_max = np.max(scores)
    scores = scores / scores_max

    # Iterate through the 4 binary scores
    for score_name, score_fn in SCORE_NAMES.items():

        # Get scoring fn
        scoring_fn = SCORE_NAMES[score_name]
        output_dic[score_name] = []

        # Iterate over outcomes
        for outc_id, outc in enumerate(outc_names):

            # Compute score for this particular outcome
            outc_labels_true = y_true[:, outc_id] == 1
            output_dic[score_name].append(scoring_fn(outc_labels_true.astype(int), scores))

    # Return object
    return output_dic


def compute_cluster_performance(X, clus_pred, y_true):
    """
    Compute cluster performance metrics given input data X and predicted cluster probability assignments clus_pred.
    Metrics computed include a) Silhouette Score, b) Davies Bouldin Score, c) Calinski Harabasz Score and d) Purity.
    Performance is computed averaged over features.

    Params:
    - X: array-like of shape (N, T, D_f) where N is the number of patients, T the number of time steps and D_f the
    number of features (+2, the id col and the time col).
    - clus_pred: array-like of shape (N, ) with predicted label assignments.
    - y_true: array-like of shape (N, num_outcs) with one-hot encoding of true class assignments.

    Returns:
        - Dictionary of output cluster performance metrics. Includes:
            - "Silhouette": Silhouette Score computation.
            - "DBI": Davies-Bouldin Index computation.
            - "VRI": Variance-Ratio Criterion (also known as Calinski Harabasz Index).
            - "Purity": Purity Score computation.
    """

    # If not converted to categorical, then convert
    if len(clus_pred.shape) == 2:
        clus_pred = np.argmax(clus_pred, axis=1)

    # Compute the same taking average over each feature dimension
    sil_avg, dbi_avg, vri_avg = 0, 0, 0

    for feat in range(X.shape[-1]):
        sil_avg += silhouette_score(X[:, :, feat], clus_pred, metric="euclidean")
        dbi_avg += davies_bouldin_score(X[:, :, feat], clus_pred)
        vri_avg += calinski_harabasz_score(X[:, :, feat], clus_pred)

    # Compute Purity Score
    purity_score = purity(y_true, clus_pred)

    # Compute average factor
    num_feats = X.shape[-1]

    # Return Dictionary
    clus_perf_dic = {
        "Silhouette": sil_avg / num_feats,
        "DBI": dbi_avg / num_feats,
        "VRI": vri_avg / num_feats,
        "Purity": purity_score / num_feats
    }

    return clus_perf_dic


In [ ]:

from csv import writer

import pandas as pd

def evaluate(y_true=None, y_pred=None, clus_pred=None, data_info=None, save_fd=None, avg=None, scores=None,
             **kwargs):
    """
    Evaluate function to print result information given results and/or experiment ids. Returns a dictionary of scores
    given the outputs of the model (for e.g. if the model does not do clustering, then no cluster scores are returned).

    Params:
    - y_true: array-like of shape (N, num_outcs) with one-hot encodings of true class membership. defaults to None.
    - y_pred: array-like of shape (N, num_outcs) with predicted outcome likelihood assignments. defaults to None.
    - clus_pred: array-like of shape (N, num_clus) with predicted cluster membership probabilities. default to None
    - data_info: dict of input data information and objects.
    - save_fd: str, folder where to write scores to.
    - age: str, useful how to average class individual scores (defaults to None, which returns no average).
    - scores: array-like of shape (N, ) of scores. Only relevant for score-based benchmarks, such as NEWS2 and/or ESI.
    - **kwargs: other parameters given to scoring supervised scores.


    Returns:
        - list of supervised performance scores and cluster performance scores (where valid).
        - prints information for each of the associated score.
        - saves score information in relevant folder.
    """
    # Checks for instances Df vs array and loads data properties
    if isinstance(y_true, pd.DataFrame):
        y_true = y_true.values

    if "news" in save_fd.lower() or "esi" in save_fd.lower():

        # Compute scores
        scores = utils.compute_from_eas_scores(y_true=y_true, scores=scores, **kwargs)

        # Definitions for completeness
        cm = None
        clus_metrics = {}

    else:

        if isinstance(y_pred, pd.DataFrame):
            y_pred = y_pred.values

        # Load data relevant properties
        data_properties = data_info["data_properties"]
        outc_names = data_properties["outc_names"]

        # Compute scores and confusion matrix
        scores, cm, Roc_curves = utils.compute_supervised_scores(y_true, y_pred, avg=avg, outc_names=outc_names)

        # Convert Confusion Matrix to pdDataFrame
        cm = pd.DataFrame(cm, index=pd.Index(data=outc_names, name="True Class"),
                          columns=pd.Index(data=outc_names, name="Predicted Class"))

        # If clustering results exist, output cluster performance scores
        clus_metrics = {}
        if clus_pred is not None:

            if isinstance(clus_pred, pd.DataFrame):
                clus_pred = clus_pred.values

            # Compute X_test in 3 dimensional format
            min_, max_ = data_properties["norm_min"], data_properties["norm_max"]
            x_test_3d = data_info["X"][-1] * (max_ - min_) + min_

            # Compute metrics
            try:
                clus_metrics = utils.compute_cluster_performance(x_test_3d, clus_pred=clus_pred, y_true=y_true)
            except ValueError:
                print("Too little predicted labels. Can't compute clustering metrics.")
                clus_metrics = {}

        # Save Confusion matrix
        cm.to_csv(save_fd + "confusion_matrix.csv", index=True, header=True)

    # Jointly compute scores
    scores = {**scores, **clus_metrics}

    # Save
    # for key, value in Roc_curves.items():
    
    #     # Get fig, ax and save
    #     fig, _ = value
    #     fig.savefig(save_fd + key)


    with open(save_fd + "scores.csv", "w+", newline="\n") as f:
        csv_writer = writer(f, delimiter=",")

        # Iterate through score key and score value(s)
        for key, value in scores.items():

            # Define row to save
            if isinstance(value, list):
                row = tuple([key, *value])
            else:
                row = tuple([key, value])

            csv_writer.writerow(row)

    # Print information
    print("\nScoring information for this experiment\n")
    for key, value in scores.items():
        print(f"{key} value: {value}")

    print("\nConfusion Matrix for predicting results", cm, sep="\n")

    return scores
